In [1]:
import cupy as cp
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import scipy.stats as st
import pickle
import optuna
import re
import gc

from lightgbm import LGBMClassifier

from sklearn import metrics
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.impute import SimpleImputer
from sklearn_pandas import DataFrameMapper
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.metrics import roc_auc_score
from sklearn.datasets import make_moons
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import mutual_info_classif
from sklearn.experimental import enable_iterative_imputer

from keras.layers import Dense, Dropout
from keras.models import Sequential

from six import StringIO

from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import math
from scipy import stats
import numpy as np
import pandas as pd
import tensorflow as tf

from datetime import datetime
from tensorflow import feature_column
from tensorflow.keras import layers

from tqdm import tqdm

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb
from sklearn.metrics import log_loss
from xgboost.sklearn import XGBClassifier

from Modules import Modules
from Baseline import Baseline
from FeatureEngineering2 import FeatureEngineering2


%matplotlib inline

print("OK\n")

OK



## 共通関数定義

In [2]:
def geometric_mean(x):
    return np.exp(np.log(x[x>0]).mean())

In [3]:
"""
params_base = {
            'boosting_type': 'gbdt'
            ,'objective': 'binary'
            ,'metric': 'auc'
            ,'verbosity': -1
            ,'learning_rate': 0.05
            ,'n_estimators': 100000
            ,'random_state':123
            ,'bagging_freq': 1   
}
"""
params_base = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_samples': 165
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}

def objective(trial):
    # 探索するパラメータ
    param_tuning = {
            "num_leaves": trial.suggest_int("num_leaves", 8, 256)
            ,"min_child_samples": trial.suggest_int("min_child_samples", 5, 200)
            ,"min_sum_hessian_in_leaf": trial.suggest_float("min_sum_hessian_in_leaf", 1e-5, 1e-2, log=True)
            ,"feature_fraction": trial.suggest_float("feature_fraction", 0.5, 1.0)
            ,"bagging_fraction": trial.suggest_float("bagging_fraction", 0.5, 1.0)
            ,"lambda_l1": trial.suggest_float("lambda_l1", 1e-2, 1e+2, log=True)
            ,"lambda_l2": trial.suggest_float("lambda_l2", 1e-2, 1e+2, log=True)
    }
    param_tuning.update(params_base)

    # モデル学習・評価:
    list_metrics = []
    cv = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=123).split(x_train, y_train))
    list_fold = [0]
    for nfold in list_fold:
        idx_tr, idx_va = cv[nfold][0], cv[nfold][1]
        x_tr, y_tr = x_train.loc[idx_tr, :], y_train[idx_tr]
        x_va, y_va = x_train.loc[idx_va, :], y_train[idx_va]
        model = lgb.LGBMClassifier(**params)
        model.fit(x_tr
                    ,y_tr
                    ,eval_set=[(x_tr, y_tr), (x_va, y_va)]
                    ,early_stopping_rounds=100
                    ,verbose=0
        )
        y_va_pred = model.predict_proba(x_va)[:,1]
        metric_va = roc_auc_score(y_va, y_va_pred)
        list_metrics.append(metric_va)

    # 評価指数の算出
    metrics = np.mean(list_metrics)

    return metrics

## インスタンス生成

In [4]:
baseline = Baseline('SK_ID_CURR')
modules = Modules('SK_ID_CURR')
FeatureEngineering2 = FeatureEngineering2('SK_ID_CURR')

### app特徴量作成

In [5]:
df = pd.read_csv(r'./home-credit-default-risk/application_train.csv')
test_df = pd.read_csv(r'./home-credit-default-risk/application_test.csv')
df = df.append(test_df).reset_index()

docs = [_f for _f in df.columns if 'FLAG_DOC' in _f]
live = [_f for _f in df.columns if ('FLAG_' in _f) & ('FLAG_DOC' not in _f) & ('_FLAG_' not in _f)]

# general cleaning procedures
df = df[df['CODE_GENDER'] != 'XNA']
df = df[df['AMT_INCOME_TOTAL'] < 20000000] # remove a outlier 117M
# NaN values for DAYS_EMPLOYED: 365.243 -> nan
df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True) # set null value
df['DAYS_LAST_PHONE_CHANGE'].replace(0, np.nan, inplace=True) # set null value

### CODE_GENDER, 
# Categorical features with Binary encode (0 or 1; two categories)
for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
    df[bin_feature], uniques = pd.factorize(df[bin_feature])

### NAME_INCOME_TYPE
df['NAME_INCOME_TYPE'].replace(['Maternity leave'], ['Unemployed'], inplace=True)
df['NAME_INCOME_TYPE'].replace(['Working','Commercial associate','Pensioner','State servant','Unemployed','Student','Businessman','Maternity leave'], [0, 1, 2, 3, 4, 5, 6, 7], inplace=True)
### NAME_FAMILY_STATUS
df['NAME_FAMILY_STATUS'].replace('Unknown', 'Married', inplace=True)
df['NAME_FAMILY_STATUS'].replace(['Married','Single / not married','Civil marriage','Separated','Widow'], [0, 1, 2, 3, 4], inplace=True)

### 
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '_', x))

### 列名から不正な文字を削除
for col in df.columns:
    df[col] = df[col].replace('\s', '_', regex=True)
    df[col] = df[col].replace(':', '_', regex=True)
    df[col] = df[col].replace('/', '_', regex=True)


# nullに置き換え
df['DAYS_EMPLOYED'] = df['DAYS_EMPLOYED'].replace(365243, np.nan)

inc_by_org = df[['AMT_INCOME_TOTAL', 'ORGANIZATION_TYPE']].groupby('ORGANIZATION_TYPE').median()['AMT_INCOME_TOTAL']

# 特徴量1: 総所得金額を世帯人数で除した値
df['INCOME_div_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']

# 特徴量2: 総所得額を就労期間で除した値
df['INCOME_div_EMPLOYED'] = df['AMT_INCOME_TOTAL'] / df['DAYS_EMPLOYED']

#
df['NEW_CREDIT_TO_ANNUITY_RATIO'] = df['AMT_CREDIT'] / df['AMT_ANNUITY']

df['NEW_CREDIT_TO_GOODS_RATIO'] = df['AMT_CREDIT'] / df['AMT_GOODS_PRICE']

df['NEW_DOC_IND_KURT'] = df[docs].kurtosis(axis=1)

df['NEW_LIVE_IND_SUM'] = df[live].sum(axis=1)

df['NEW_INC_PER_CHLD'] = df['AMT_INCOME_TOTAL'] / (1 + df['CNT_CHILDREN'])

df['NEW_INC_BY_ORG'] = df['ORGANIZATION_TYPE'].map(inc_by_org)

df['NEW_EMPLOY_TO_BIRTH_RATIO'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']

df['NEW_EMPLOY_TO_BIRTH-18_RATIO'] = df['DAYS_EMPLOYED'] / (df['DAYS_BIRTH'] + 18*365)

df['NEW_BIRTH_TO_EMPLOY_RATIO'] = df['DAYS_BIRTH'] / (1 + df['DAYS_EMPLOYED'])

df['NEW_INCOME_TO_ANNUITY_RATIO'] = df['AMT_INCOME_TOTAL'] / (1 + df['AMT_ANNUITY'])

df['NEW_ANNUITY_TO_INCOME_RATIO'] = df['AMT_ANNUITY'] / (1 + df['AMT_INCOME_TOTAL'])


# 特徴量3: 外部スコアの平均
df['NEW_EXT_SOURCES_MEDIAN'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].median(axis=1, skipna=True)

df['EXT_SOURCE_mean'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
    
df['EXT_SOURCE_max'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].max(axis=1)

df['EXT_SOURCE_min'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].min(axis=1)

df['EXT_SOURCE_std'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)

df['EXT_SOURCE_count'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].notnull().sum(axis=1)

df['NEW_EXT_SOURCES_PROD'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].prod(axis=1, skipna=True, min_count=1)

df['NEW_EXT_SOURCES_MAD'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mad(axis=1, skipna=True)

df['NEW_EXT_SOURCES_GEO'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].apply(geometric_mean, axis=1)

# 特徴量4: 就労期間を年齢で除した値
df['DAYS_EMPLOYED_div_BERTH'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']

# 特徴量5: 年金支払金額を所得金額で除した値
df['ANNUITY_div_INCOME'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']

# 特徴量6: 年金支払額を借入金で除した値
df['ANNUITY_div_CREDIT'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']

### 
df['credit_goods_price_ratio'] = df['AMT_CREDIT'] / df['AMT_GOODS_PRICE']
 
df['credit_downpayment'] = df['AMT_GOODS_PRICE'] - df['AMT_CREDIT']

df['USER_AGE'] = abs(df['DAYS_BIRTH'])/364.25

df['NEW_CAR_TO_BIRTH_RATIO'] = df['OWN_CAR_AGE'] / df['DAYS_BIRTH']

df['NEW_CAR_TO_EMPLOY_RATIO'] = df['OWN_CAR_AGE'] / df['DAYS_EMPLOYED']

df['NEW_PHONE_TO_BIRTH_RATIO'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_BIRTH']

df['NEW_PHONE_TO_EMPLOYED_RATIO'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_EMPLOYED']

df['NEW_CREDIT_TO_INCOME_RATIO'] = df['AMT_CREDIT'] / df['AMT_INCOME_TOTAL']

df['NEW_INCOME_CREDIT_PERC'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']

df['NEW_INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']

### 2022/06/25
df['app_EXT_SOURCE_prod'] = df['EXT_SOURCE_1'] * df['EXT_SOURCE_2'] * df['EXT_SOURCE_3']
df['app_EXT_SOURCE_1_times_EXT_SOURCE_2'] = df['EXT_SOURCE_1'] * df['EXT_SOURCE_2']
df['app_EXT_SOURCE_1_times_EXT_SOURCE_3'] = df['EXT_SOURCE_1'] * df['EXT_SOURCE_3']
df['app_EXT_SOURCE_2_times_EXT_SOURCE_3'] = df['EXT_SOURCE_2'] * df['EXT_SOURCE_3']
df['app_EXT_SOURCE_1_times_DAYS_EMPLOYED'] = df['EXT_SOURCE_1'] * df['DAYS_EMPLOYED']
df['app_EXT_SOURCE_2_times_DAYS_EMPLOYED'] = df['EXT_SOURCE_2'] * df['DAYS_EMPLOYED']
df['app_EXT_SOURCE_3_times_DAYS_EMPLOYED'] = df['EXT_SOURCE_3'] * df['DAYS_EMPLOYED']
df['app_EXT_SOURCE_1_PER_DAYS_BIRTH'] = df['EXT_SOURCE_1'] / df['DAYS_BIRTH']
df['app_EXT_SOURCE_2_PER_DAYS_BIRTH'] = df['EXT_SOURCE_2'] / df['DAYS_BIRTH']
df['app_EXT_SOURCE_3_PER_DAYS_BIRTH'] = df['EXT_SOURCE_3'] / df['DAYS_BIRTH']

### 2022/06/26
df['app_most_popular_AMT_GOODS_PRICE'] = df['AMT_GOODS_PRICE'] \
    .isin([225000, 450000, 675000, 900000]).map({True: 1, False: 0})
df['app_popular_AMT_GOODS_PRICE'] = df['AMT_GOODS_PRICE'] \
    .isin([1125000, 1350000, 1575000, 1800000, 2250000]).map({True: 1, False: 0})
df['app_OWN_CAR_AGE_div_DAYS_BIRTH'] = df['OWN_CAR_AGE'] / df['DAYS_BIRTH']
df['app_OWN_CAR_AGE_div_DAYS_EMPLOYED'] = df['OWN_CAR_AGE'] / df['DAYS_EMPLOYED']
df['app_DAYS_LAST_PHONE_CHANGE_div_DAYS_BIRTH'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_BIRTH']
df['app_DAYS_LAST_PHONE_CHANGE_div_DAYS_EMPLOYED'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_EMPLOYED']
df['app_DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED'] - df['DAYS_BIRTH']
df['app_DAYS_EMPLOYED_div_DAYS_BIRTH'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
df['app_CNT_CHILDREN_div_CNT_FAM_MEMBERS'] = df['CNT_CHILDREN'] / df['CNT_FAM_MEMBERS']

### 2022/06/29
#### EXT_SOURCE_X FEATURE
#df['APP_SCORE1_TO_BIRTH_RATIO'] = df['EXT_SOURCE_1'] / (df['DAYS_BIRTH'] / 365.25)
#df['APP_SCORE2_TO_BIRTH_RATIO'] = df['EXT_SOURCE_2'] / (df['DAYS_BIRTH'] / 365.25)
#df['APP_SCORE3_TO_BIRTH_RATIO'] = df['EXT_SOURCE_3'] / (df['DAYS_BIRTH'] / 365.25)
#df['APP_SCORE1_TO_EMPLOY_RATIO'] = df['EXT_SOURCE_1'] / (df['DAYS_EMPLOYED'] / 365.25)
#### Time ratios
#df['ID_TO_BIRTH_RATIO'] = df['DAYS_ID_PUBLISH'] / df['DAYS_BIRTH']

### one-hotencoding
encoding_columns = []

#### one-hotencoding処理対象列を取得
for col in df.columns:
    if df[col].dtype=="O":
        print(col)
        encoding_columns.append(col)

#### one-hotencoding処理対象列を追加
encoding_columns.append('CODE_GENDER')
encoding_columns.append('FLAG_OWN_CAR')
encoding_columns.append('FLAG_OWN_REALTY')
encoding_columns.append('NAME_INCOME_TYPE')
encoding_columns.append('NAME_FAMILY_STATUS')

#### one-hotencoding実行
for col in encoding_columns:
    df = pd.get_dummies(df, columns=[col], dummy_na=True)

### 列名の不正な文字を置換(ドット対応)
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '_', x))

### 極大値、極小値をNULLへ置換
df = df.replace([np.inf, -np.inf], np.nan)

len(df.columns)

df.head()

NAME_CONTRACT_TYPE
NAME_TYPE_SUITE
NAME_EDUCATION_TYPE
NAME_HOUSING_TYPE
OCCUPATION_TYPE
WEEKDAY_APPR_PROCESS_START
ORGANIZATION_TYPE
FONDKAPREMONT_MODE
HOUSETYPE_MODE
WALLSMATERIAL_MODE
EMERGENCYSTATE_MODE


,index,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,NAME_INCOME_TYPE_4_0,NAME_INCOME_TYPE_5_0,NAME_INCOME_TYPE_6_0,NAME_INCOME_TYPE_nan,NAME_FAMILY_STATUS_0_0,NAME_FAMILY_STATUS_1_0,NAME_FAMILY_STATUS_2_0,NAME_FAMILY_STATUS_3_0,NAME_FAMILY_STATUS_4_0,NAME_FAMILY_STATUS_nan
0,0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,...,0,0,0,0,0,1,0,0,0,0
1,1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,...,0,0,0,0,1,0,0,0,0,0
2,2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,...,0,0,0,0,0,1,0,0,0,0
3,3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,...,0,0,0,0,0,0,1,0,0,0
4,4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,...,0,0,0,0,0,1,0,0,0,0


In [6]:
### indexを削除
df=df.reset_index()
df.head()

,level_0,index,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,NAME_INCOME_TYPE_4_0,NAME_INCOME_TYPE_5_0,NAME_INCOME_TYPE_6_0,NAME_INCOME_TYPE_nan,NAME_FAMILY_STATUS_0_0,NAME_FAMILY_STATUS_1_0,NAME_FAMILY_STATUS_2_0,NAME_FAMILY_STATUS_3_0,NAME_FAMILY_STATUS_4_0,NAME_FAMILY_STATUS_nan
0,0,0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018801,...,0,0,0,0,0,1,0,0,0,0
1,1,1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,...,0,0,0,0,1,0,0,0,0,0
2,2,2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010032,...,0,0,0,0,0,1,0,0,0,0
3,3,3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008019,...,0,0,0,0,0,0,1,0,0,0
4,4,4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028663,...,0,0,0,0,0,1,0,0,0,0


In [7]:
### indexを削除
df = df.drop(columns=['index'])
df.head()

,level_0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,NAME_INCOME_TYPE_4_0,NAME_INCOME_TYPE_5_0,NAME_INCOME_TYPE_6_0,NAME_INCOME_TYPE_nan,NAME_FAMILY_STATUS_0_0,NAME_FAMILY_STATUS_1_0,NAME_FAMILY_STATUS_2_0,NAME_FAMILY_STATUS_3_0,NAME_FAMILY_STATUS_4_0,NAME_FAMILY_STATUS_nan
0,0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,...,0,0,0,0,0,1,0,0,0,0
1,1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,...,0,0,0,0,1,0,0,0,0,0
2,2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,...,0,0,0,0,0,1,0,0,0,0
3,3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,...,0,0,0,0,0,0,1,0,0,0
4,4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,...,0,0,0,0,0,1,0,0,0,0


In [8]:
df = baseline.reduce_mem_usage(df)
print(df.shape)
df.head()

Memory usage of dataframe is 489.57 MB
Memory usage of optimization is 403275128.00 MB
Decreased by -82372426.9%
(356250, 314)


,level_0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,NAME_INCOME_TYPE_4_0,NAME_INCOME_TYPE_5_0,NAME_INCOME_TYPE_6_0,NAME_INCOME_TYPE_nan,NAME_FAMILY_STATUS_0_0,NAME_FAMILY_STATUS_1_0,NAME_FAMILY_STATUS_2_0,NAME_FAMILY_STATUS_3_0,NAME_FAMILY_STATUS_4_0,NAME_FAMILY_STATUS_nan
0,0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [9]:
for col in df.columns:
    print(col)

level_0
SK_ID_CURR
TARGET
CNT_CHILDREN
AMT_INCOME_TOTAL
AMT_CREDIT
AMT_ANNUITY
AMT_GOODS_PRICE
REGION_POPULATION_RELATIVE
DAYS_BIRTH
DAYS_EMPLOYED
DAYS_REGISTRATION
DAYS_ID_PUBLISH
OWN_CAR_AGE
FLAG_MOBIL
FLAG_EMP_PHONE
FLAG_WORK_PHONE
FLAG_CONT_MOBILE
FLAG_PHONE
FLAG_EMAIL
CNT_FAM_MEMBERS
REGION_RATING_CLIENT
REGION_RATING_CLIENT_W_CITY
HOUR_APPR_PROCESS_START
REG_REGION_NOT_LIVE_REGION
REG_REGION_NOT_WORK_REGION
LIVE_REGION_NOT_WORK_REGION
REG_CITY_NOT_LIVE_CITY
REG_CITY_NOT_WORK_CITY
LIVE_CITY_NOT_WORK_CITY
EXT_SOURCE_1
EXT_SOURCE_2
EXT_SOURCE_3
APARTMENTS_AVG
BASEMENTAREA_AVG
YEARS_BEGINEXPLUATATION_AVG
YEARS_BUILD_AVG
COMMONAREA_AVG
ELEVATORS_AVG
ENTRANCES_AVG
FLOORSMAX_AVG
FLOORSMIN_AVG
LANDAREA_AVG
LIVINGAPARTMENTS_AVG
LIVINGAREA_AVG
NONLIVINGAPARTMENTS_AVG
NONLIVINGAREA_AVG
APARTMENTS_MODE
BASEMENTAREA_MODE
YEARS_BEGINEXPLUATATION_MODE
YEARS_BUILD_MODE
COMMONAREA_MODE
ELEVATORS_MODE
ENTRANCES_MODE
FLOORSMAX_MODE
FLOORSMIN_MODE
LANDAREA_MODE
LIVINGAPARTMENTS_MODE
LIVINGAREA_MODE


In [10]:
app_train = df[df['TARGET'].notnull()].copy()
app_test = df[df['TARGET'].isnull()].copy()
app_test = app_test.drop(columns=['TARGET'])

print(app_test.shape)
df.head()

(48744, 313)


,level_0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,NAME_INCOME_TYPE_4_0,NAME_INCOME_TYPE_5_0,NAME_INCOME_TYPE_6_0,NAME_INCOME_TYPE_nan,NAME_FAMILY_STATUS_0_0,NAME_FAMILY_STATUS_1_0,NAME_FAMILY_STATUS_2_0,NAME_FAMILY_STATUS_3_0,NAME_FAMILY_STATUS_4_0,NAME_FAMILY_STATUS_nan
0,0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [11]:
app_train = baseline.reduce_mem_usage(app_train)
app_test = baseline.reduce_mem_usage(app_test)
print(app_test.shape)
app_test.head()

Memory usage of dataframe is 334.32 MB
Memory usage of optimization is 350556840.00 MB
Decreased by -104857500.0%
Memory usage of dataframe is 52.81 MB
Memory usage of optimization is 55373184.00 MB
Decreased by -104857500.0%
(48744, 313)


,level_0,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_INCOME_TYPE_4_0,NAME_INCOME_TYPE_5_0,NAME_INCOME_TYPE_6_0,NAME_INCOME_TYPE_nan,NAME_FAMILY_STATUS_0_0,NAME_FAMILY_STATUS_1_0,NAME_FAMILY_STATUS_2_0,NAME_FAMILY_STATUS_3_0,NAME_FAMILY_STATUS_4_0,NAME_FAMILY_STATUS_nan
307506,307511,100001,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
307507,307512,100005,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
307508,307513,100013,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
307509,307514,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
307510,307515,100038,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [12]:
app_train.head()

,level_0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,NAME_INCOME_TYPE_4_0,NAME_INCOME_TYPE_5_0,NAME_INCOME_TYPE_6_0,NAME_INCOME_TYPE_nan,NAME_FAMILY_STATUS_0_0,NAME_FAMILY_STATUS_1_0,NAME_FAMILY_STATUS_2_0,NAME_FAMILY_STATUS_3_0,NAME_FAMILY_STATUS_4_0,NAME_FAMILY_STATUS_nan
0,0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [13]:
x_train = app_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = app_train['TARGET']
id_train = app_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=='O':
        x_train[col] = x_train[col].astype('category')

In [14]:
baseline = Baseline('SK_ID_CURR')

"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.05
    ,'num_leaves': 32
    ,'n_estimators': 100000
    ,'random_state':123
    ,'importance_type': 'gain'
}
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    #,'device_type': 'gpu'
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'num_leaves': 58
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_weight': 6
    ,'min_child_samples': 165
    #,'keep_training_booster': True
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}

train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246004, 312) (61502, 312)
[LightGBM] [Warning] num_threads is set with n_jobs=-1, nthread=-1 will be ignored. Current value: num_threads=-1
[100]	training's auc: 0.759217	valid_1's auc: 0.744259
[200]	training's auc: 0.770645	valid_1's auc: 0.751871
[300]	training's auc: 0.780938	valid_1's auc: 0.757313
[400]	training's auc: 0.79025	valid_1's auc: 0.761884
[500]	training's auc: 0.797876	valid_1's auc: 0.764523
[600]	training's auc: 0.804576	valid_1's auc: 0.766249
[700]	training's auc: 0.810585	valid_1's auc: 0.767559
[800]	training's auc: 0.81626	valid_1's auc: 0.768437
[900]	training's auc: 0.821506	valid_1's auc: 0.769211
[1000]	training's auc: 0.826354	valid_1's auc: 0.769698
[1100]	training's auc: 0.831112	valid_1's auc: 0.770014
[1200]	training's auc: 0.835506	valid_1's auc: 0.770234
[1300]	training's auc: 0.839889	valid_1's auc: 0.770462
[1400]	training's auc: 0.844023	valid_1's auc: 0.770518
[1500]	training's auc: 0.848028	valid_1's 

In [15]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
48,EXT_SOURCE_mean,331971.567997,18771.893434
160,NEW_EXT_SOURCES_GEO,157662.372390,19703.916844
154,NEW_CREDIT_TO_ANNUITY_RATIO,67419.006323,1925.305204
162,NEW_EXT_SOURCES_MEDIAN,61744.984181,10721.306062
10,ANNUITY_div_CREDIT,46700.512096,2733.098096
45,EXT_SOURCE_3,27432.025223,1494.348750
47,EXT_SOURCE_max,26683.261575,2766.255220
301,app_EXT_SOURCE_2_times_EXT_SOURCE_3,25822.788961,5071.475174
155,NEW_CREDIT_TO_GOODS_RATIO,24364.195729,689.892938
295,app_EXT_SOURCE_1_PER_DAYS_BIRTH,21841.784029,528.648557


In [16]:
# データセット
x_test = app_test.drop(columns=['SK_ID_CURR'])
id_test = app_test[['SK_ID_CURR']]

for col in x_test.columns:
    if x_test[col].dtype=='O':
        x_test[col] = x_test[col].astype('category')

In [17]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [18]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_featureEngineering1.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(97488, 2)


,SK_ID_CURR,TARGET
0,NaN,0.031613
1,NaN,0.072152
2,NaN,0.015154
3,NaN,0.037028
4,NaN,0.153376


### CSVの読み込みと前処理

In [19]:
# infをnanへ置換
#app_train = app_train.replace([np.inf, -np.inf], np.nan)
#app_train.head()

In [20]:
# 欠損値補完
#app_train = app_train.fillna(-99999999)
#app_train.head()

In [21]:
# infをnanへ置換
#app_test = app_test.replace([np.inf, -np.inf], np.nan)
#app_test.head()

In [22]:
# 欠損値補完
#app_test = app_test.fillna(-99999999)
#app_test.head()

In [23]:
#app_train_origin = pd.read_csv('./home-credit-default-risk/exports/app_train.csv')
#app_test_origin =  pd.read_csv('./home-credit-default-risk/exports/app_test.csv')
bureau_origin = pd.read_csv('./home-credit-default-risk/bureau.csv')
bureau_balance_origin = pd.read_csv('./home-credit-default-risk/bureau_balance.csv')
pos_cash_balance_origin = pd.read_csv('./home-credit-default-risk/POS_CASH_balance.csv')
credit_card_balance_origin = pd.read_csv('./home-credit-default-risk/credit_card_balance.csv')
previous_application_origin = pd.read_csv('./home-credit-default-risk/previous_application.csv')
installments_payments_origin = pd.read_csv('./home-credit-default-risk/installments_payments.csv')
# custom data
#app_train_loan_from_hc_only_cash_origin = pd.read_csv('./home-credit-default-risk/datasets/app_train_loan_from_hc_only_cash.csv')
#app_test_loan_from_hc_only_cash_origin = pd.read_csv('./home-credit-default-risk/datasets/app_test_loan_from_hc_only_cash.csv')
#app_train_1000_mean_origin = pd.read_csv('./home-credit-default-risk/exports/train_hc_only_cash_installment_payment_ratio_1000_mean_mean.csv')
#app_test_1000_mean_origin = pd.read_csv('./home-credit-default-risk/exports/test_hc_only_cash_installment_payment_ratio_1000_mean_mean.csv')
#app_train_max_installment_ratio_origin = pd.read_csv('./home-credit-default-risk/exports/train_hc_annuity_to_max_installment_ratio.csv')
#app_test_max_installment_ratio_origin = pd.read_csv('./home-credit-default-risk/exports/test_hc_annuity_to_max_installment_ratio.csv')

In [24]:
#app_train = app_train_origin.copy()
#app_test = app_test_origin.copy()
bureau = bureau_origin.copy()
bureau_balance = bureau_balance_origin.copy()
pos_cash_balance = pos_cash_balance_origin.copy()
credit_card_balance = credit_card_balance_origin.copy()
previous_application = previous_application_origin.copy()
installments_payments = installments_payments_origin.copy()
#app_train_1000_mean = app_train_1000_mean_origin.copy()
#app_test_1000_mean = app_test_1000_mean_origin.copy()
#app_train_max_installment_ratio = app_train_max_installment_ratio_origin.copy()
#app_test_max_installment_ratio = app_test_max_installment_ratio_origin.copy()
pos_cash_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [25]:
# infをnanへ置換
bureau = bureau.replace([np.inf, -np.inf], np.nan)
bureau_balance = bureau_balance.replace([np.inf, -np.inf], np.nan)
pos_cash_balance = pos_cash_balance.replace([np.inf, -np.inf], np.nan)
credit_card_balance = credit_card_balance.replace([np.inf, -np.inf], np.nan)
previous_application = previous_application.replace([np.inf, -np.inf], np.nan)
installments_payments = installments_payments.replace([np.inf, -np.inf], np.nan)

pos_cash_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


### pos_cashの紐づけ

In [26]:
baseline = Baseline('SK_ID_CURR')

pos = pos_cash_balance.copy()
pos = baseline.reduce_mem_usage(pos)
print(pos.shape)
pos.head()

Memory usage of dataframe is 610.43 MB
Memory usage of optimization is 290039510.00 MB
Decreased by -47513511.5%
(10001358, 8)


,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [27]:
pos_ohe = pd.get_dummies(pos, columns=['NAME_CONTRACT_STATUS'], dummy_na=True)
col_ohe = sorted(list(set(pos_ohe.columns) - set(pos.columns)))
print(len(col_ohe))
col_ohe

10


['NAME_CONTRACT_STATUS_Active',
 'NAME_CONTRACT_STATUS_Amortized debt',
 'NAME_CONTRACT_STATUS_Approved',
 'NAME_CONTRACT_STATUS_Canceled',
 'NAME_CONTRACT_STATUS_Completed',
 'NAME_CONTRACT_STATUS_Demand',
 'NAME_CONTRACT_STATUS_Returned to the store',
 'NAME_CONTRACT_STATUS_Signed',
 'NAME_CONTRACT_STATUS_XNA',
 'NAME_CONTRACT_STATUS_nan']

In [28]:
pos_ohe_agg = pos_ohe.groupby('SK_ID_CURR').agg({
    "MONTHS_BALANCE": ["mean", "std", "min", "max"]
    ,"CNT_INSTALMENT": ["mean", "std", "min", "max"]
    ,"CNT_INSTALMENT_FUTURE": ["mean", "std", "min", "max"]
    ,"SK_DPD": ["mean", "std", "min", "max"]
    ,"SK_DPD_DEF": ["mean", "std", "min", "max"]
    # カテゴリ変数をone-hot-encodingした値の集約
    ,"NAME_CONTRACT_STATUS_Active": ["mean"]
    ,"NAME_CONTRACT_STATUS_Amortized debt": ["mean"]
    ,"NAME_CONTRACT_STATUS_Approved": ["mean"]
    ,"NAME_CONTRACT_STATUS_Canceled": ["mean"]
    ,"NAME_CONTRACT_STATUS_Completed": ["mean"]
    ,"NAME_CONTRACT_STATUS_Demand": ["mean"]
    ,"NAME_CONTRACT_STATUS_Returned to the store": ["mean"]
    ,"NAME_CONTRACT_STATUS_Signed": ["mean"]
    ,"NAME_CONTRACT_STATUS_XNA": ["mean"]
    ,"NAME_CONTRACT_STATUS_nan": ["mean"]
    # IDのユニーク数をカウント
    ,"SK_ID_PREV":["count", "nunique"]
})

# カラム名の付与
pos_ohe_agg.columns = [i + "_" + j for i,j in pos_ohe_agg.columns]
pos_ohe_agg = pos_ohe_agg.reset_index(drop=False)

print(pos_ohe_agg.shape)
pos_ohe_agg.head()

(337252, 33)


,SK_ID_CURR,MONTHS_BALANCE_mean,MONTHS_BALANCE_std,MONTHS_BALANCE_min,MONTHS_BALANCE_max,CNT_INSTALMENT_mean,CNT_INSTALMENT_std,CNT_INSTALMENT_min,CNT_INSTALMENT_max,CNT_INSTALMENT_FUTURE_mean,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100001,-72.555556,20.863312,-96,-53,4.000000,0.000000,4.0,4.0,1.444444,...,0.0,0.0,0.222222,0.0,0.0,0.000000,0.0,0.0,9,2
1,100002,-10.000000,5.627314,-19,-1,24.000000,0.000000,24.0,24.0,15.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,19,1
2,100003,-43.785714,24.640162,-77,-18,10.107142,2.806597,6.0,12.0,5.785714,...,0.0,0.0,0.071429,0.0,0.0,0.000000,0.0,0.0,28,3
3,100004,-25.500000,1.290994,-27,-24,3.750000,0.500000,3.0,4.0,2.250000,...,0.0,0.0,0.250000,0.0,0.0,0.000000,0.0,0.0,4,1
4,100005,-20.000000,3.316625,-25,-15,11.700000,0.948683,9.0,12.0,7.200000,...,0.0,0.0,0.090909,0.0,0.0,0.090909,0.0,0.0,11,1


In [29]:
### infをnanへ置換
pos_ohe_agg = pos_ohe_agg.replace([np.inf, -np.inf], np.nan)
pos_ohe_agg.head()

,SK_ID_CURR,MONTHS_BALANCE_mean,MONTHS_BALANCE_std,MONTHS_BALANCE_min,MONTHS_BALANCE_max,CNT_INSTALMENT_mean,CNT_INSTALMENT_std,CNT_INSTALMENT_min,CNT_INSTALMENT_max,CNT_INSTALMENT_FUTURE_mean,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100001,-72.555556,20.863312,-96,-53,4.000000,0.000000,4.0,4.0,1.444444,...,0.0,0.0,0.222222,0.0,0.0,0.000000,0.0,0.0,9,2
1,100002,-10.000000,5.627314,-19,-1,24.000000,0.000000,24.0,24.0,15.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,19,1
2,100003,-43.785714,24.640162,-77,-18,10.107142,2.806597,6.0,12.0,5.785714,...,0.0,0.0,0.071429,0.0,0.0,0.000000,0.0,0.0,28,3
3,100004,-25.500000,1.290994,-27,-24,3.750000,0.500000,3.0,4.0,2.250000,...,0.0,0.0,0.250000,0.0,0.0,0.000000,0.0,0.0,4,1
4,100005,-20.000000,3.316625,-25,-15,11.700000,0.948683,9.0,12.0,7.200000,...,0.0,0.0,0.090909,0.0,0.0,0.090909,0.0,0.0,11,1


In [30]:
df_train = pd.merge(app_train, pos_ohe_agg, on='SK_ID_CURR', how='left')
print(df_train.shape)
df_train.head()

(307506, 346)


,level_0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,19.0,1.0
1,1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,...,0.0,0.0,0.071429,0.0,0.000000,0.000000,0.0,0.0,28.0,3.0
2,2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,...,0.0,0.0,0.250000,0.0,0.000000,0.000000,0.0,0.0,4.0,1.0
3,3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,...,0.0,0.0,0.095238,0.0,0.047619,0.000000,0.0,0.0,21.0,3.0
4,4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,...,0.0,0.0,0.045455,0.0,0.000000,0.015152,0.0,0.0,66.0,5.0


In [31]:
### テーブル結合
df_test = pd.merge(app_test, pos_ohe_agg, on='SK_ID_CURR', how="left")
print(df_test.shape)
df_test.head()

(48744, 345)


,level_0,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,307511,100001,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,...,0.0,0.0,0.222222,0.0,0.0,0.000000,0.0,0.0,9.0,2.0
1,307512,100005,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,...,0.0,0.0,0.090909,0.0,0.0,0.090909,0.0,0.0,11.0,1.0
2,307513,100013,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,...,0.0,0.0,0.083333,0.0,0.0,0.027778,0.0,0.0,36.0,3.0
3,307514,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,...,0.0,0.0,0.064516,0.0,0.0,0.000000,0.0,0.0,31.0,2.0
4,307515,100038,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,...,0.0,0.0,0.076923,0.0,0.0,0.000000,0.0,0.0,13.0,1.0


In [32]:
### 処理対象の列名を配列化
#encoding_columns = []
encoding_columns = list(df_test.select_dtypes(include='number').columns)

#for col in df_test.columns:
#    print(col)
#    encoding_columns.append(col)
    
encoding_columns.remove('SK_ID_CURR')

In [33]:
### Yao-Jhonson変換
for column in encoding_columns:
    modules.process_yj_convert(df_train, df_test, column, [column])

df_train.head()

,level_0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,-2.196836,100002,1.0,-0.653131,0.647085,-0.284046,0.026932,-0.324380,0.047085,1.597501,...,-0.117324,-0.006439,-1.882714,-0.03285,-0.122221,-0.493724,-0.002629,0.0,-0.187757,-1.255497
1,-2.196523,100003,0.0,-0.653131,1.213324,1.495396,0.744618,1.433694,-1.730864,-0.222757,...,-0.117324,-0.006439,-0.045923,-0.03285,-0.122221,-0.493724,-0.002629,0.0,0.304656,0.504269
2,-2.196256,100004,0.0,-0.653131,-1.654456,-1.648319,-2.082107,-1.533668,-0.863026,-0.716549,...,-0.117324,-0.006439,2.215187,-0.03285,-0.122221,-0.493724,-0.002629,0.0,-2.038472,-1.255497
3,-2.196014,100006,0.0,-0.653131,-0.176134,-0.635707,0.377928,-0.549543,-1.112439,-0.707904,...,-0.117324,-0.006439,0.405962,-0.03285,8.181937,-0.493724,-0.002629,0.0,-0.061302,0.504269
4,-2.195789,100007,0.0,-0.653131,-0.394983,0.042350,-0.198042,0.210360,0.799714,-0.901463,...,-0.117324,-0.006439,-0.620243,-0.03285,-0.122221,1.480653,-0.002629,0.0,1.408833,1.246159


In [34]:
### indexを設定
df_train = df_train.set_index('SK_ID_CURR')
df_test = df_test.set_index('SK_ID_CURR')
df_train.head()

,level_0,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,-2.196836,1.0,-0.653131,0.647085,-0.284046,0.026932,-0.324380,0.047085,1.597501,0.846146,...,-0.117324,-0.006439,-1.882714,-0.03285,-0.122221,-0.493724,-0.002629,0.0,-0.187757,-1.255497
100003,-2.196523,0.0,-0.653131,1.213324,1.495396,0.744618,1.433694,-1.730864,-0.222757,0.295507,...,-0.117324,-0.006439,-0.045923,-0.03285,-0.122221,-0.493724,-0.002629,0.0,0.304656,0.504269
100004,-2.196256,0.0,-0.653131,-1.654456,-1.648319,-2.082107,-1.533668,-0.863026,-0.716549,1.649642,...,-0.117324,-0.006439,2.215187,-0.03285,-0.122221,-0.493724,-0.002629,0.0,-2.038472,-1.255497
100006,-2.196014,0.0,-0.653131,-0.176134,-0.635707,0.377928,-0.549543,-1.112439,-0.707904,-0.647130,...,-0.117324,-0.006439,0.405962,-0.03285,8.181937,-0.493724,-0.002629,0.0,-0.061302,0.504269
100007,-2.195789,0.0,-0.653131,-0.394983,0.042350,-0.198042,0.210360,0.799714,-0.901463,-0.646418,...,-0.117324,-0.006439,-0.620243,-0.03285,-0.122221,1.480653,-0.002629,0.0,1.408833,1.246159


In [35]:
### 標準化
for column in encoding_columns:
    num_cols = []
    num_cols.append(column)
    result = modules.process_standardization(df_train, df_test, num_cols)
        
df_train.head()

,level_0,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,-2.196836,1.0,-0.653131,0.647085,-0.284046,0.026932,-0.324380,0.047085,1.597501,0.846146,...,-0.117324,-0.006439,-1.882714,-0.03285,-0.122221,-0.493724,-0.002629,0.0,-0.187757,-1.255497
100003,-2.196523,0.0,-0.653131,1.213324,1.495396,0.744618,1.433694,-1.730864,-0.222757,0.295507,...,-0.117324,-0.006439,-0.045923,-0.03285,-0.122221,-0.493724,-0.002629,0.0,0.304656,0.504269
100004,-2.196256,0.0,-0.653131,-1.654456,-1.648319,-2.082107,-1.533668,-0.863026,-0.716549,1.649642,...,-0.117324,-0.006439,2.215187,-0.03285,-0.122221,-0.493724,-0.002629,0.0,-2.038472,-1.255497
100006,-2.196014,0.0,-0.653131,-0.176134,-0.635707,0.377928,-0.549543,-1.112439,-0.707904,-0.647130,...,-0.117324,-0.006439,0.405962,-0.03285,8.181937,-0.493724,-0.002629,0.0,-0.061302,0.504269
100007,-2.195789,0.0,-0.653131,-0.394983,0.042350,-0.198042,0.210360,0.799714,-0.901463,-0.646418,...,-0.117324,-0.006439,-0.620243,-0.03285,-0.122221,1.480653,-0.002629,0.0,1.408833,1.246159


In [36]:
### indexを削除
df_train = df_train.reset_index()
df_test = df_test.reset_index()
df_train.head()

,SK_ID_CURR,level_0,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100002,-2.196836,1.0,-0.653131,0.647085,-0.284046,0.026932,-0.324380,0.047085,1.597501,...,-0.117324,-0.006439,-1.882714,-0.03285,-0.122221,-0.493724,-0.002629,0.0,-0.187757,-1.255497
1,100003,-2.196523,0.0,-0.653131,1.213324,1.495396,0.744618,1.433694,-1.730864,-0.222757,...,-0.117324,-0.006439,-0.045923,-0.03285,-0.122221,-0.493724,-0.002629,0.0,0.304656,0.504269
2,100004,-2.196256,0.0,-0.653131,-1.654456,-1.648319,-2.082107,-1.533668,-0.863026,-0.716549,...,-0.117324,-0.006439,2.215187,-0.03285,-0.122221,-0.493724,-0.002629,0.0,-2.038472,-1.255497
3,100006,-2.196014,0.0,-0.653131,-0.176134,-0.635707,0.377928,-0.549543,-1.112439,-0.707904,...,-0.117324,-0.006439,0.405962,-0.03285,8.181937,-0.493724,-0.002629,0.0,-0.061302,0.504269
4,100007,-2.195789,0.0,-0.653131,-0.394983,0.042350,-0.198042,0.210360,0.799714,-0.901463,...,-0.117324,-0.006439,-0.620243,-0.03285,-0.122221,1.480653,-0.002629,0.0,1.408833,1.246159


In [37]:
### メモリサイズを縮小
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()

Memory usage of dataframe is 487.99 MB
Memory usage of optimization is 425588432.00 MB
Decreased by -87213196.7%
Memory usage of dataframe is 77.17 MB
Memory usage of optimization is 67266848.00 MB
Decreased by -87170703.9%


,SK_ID_CURR,level_0,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100002,-2.197266,1.0,-0.65332,0.646973,-0.283936,0.026932,-0.324463,0.047089,1.597656,...,-0.11731,-0.006439,-1.882812,-0.032837,-0.122192,-0.493652,-0.002628,0.0,-0.187744,-1.255859
1,100003,-2.197266,0.0,-0.65332,1.212891,1.495117,0.744629,1.433594,-1.730469,-0.222778,...,-0.11731,-0.006439,-0.045929,-0.032837,-0.122192,-0.493652,-0.002628,0.0,0.304688,0.504395
2,100004,-2.195312,0.0,-0.65332,-1.654297,-1.648438,-2.082031,-1.533203,-0.862793,-0.716309,...,-0.11731,-0.006439,2.214844,-0.032837,-0.122192,-0.493652,-0.002628,0.0,-2.039062,-1.255859
3,100006,-2.195312,0.0,-0.65332,-0.176147,-0.635742,0.377930,-0.549316,-1.112305,-0.708008,...,-0.11731,-0.006439,0.406006,-0.032837,8.179688,-0.493652,-0.002628,0.0,-0.061310,0.504395
4,100007,-2.195312,0.0,-0.65332,-0.395020,0.042358,-0.197998,0.210327,0.799805,-0.901367,...,-0.11731,-0.006439,-0.620117,-0.032837,-0.122192,1.480469,-0.002628,0.0,1.409180,1.246094


In [38]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=="O":
        x_train[col] = x_train[col].astype('category')

In [39]:
baseline = Baseline('SK_ID_CURR')

"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.05
    ,'num_leaves': 32
    ,'n_estimators': 100000
    ,'random_state':123
    ,'importance_type': 'gain'
}
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    #,'device_type': 'gpu'
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'num_leaves': 58
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_weight': 6
    ,'min_child_samples': 165
    #,'keep_training_booster': True
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}

train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246004, 344) (61502, 344)
[LightGBM] [Warning] num_threads is set with n_jobs=-1, nthread=-1 will be ignored. Current value: num_threads=-1
[100]	training's auc: 0.764512	valid_1's auc: 0.746438
[200]	training's auc: 0.777517	valid_1's auc: 0.754796
[300]	training's auc: 0.788828	valid_1's auc: 0.760801
[400]	training's auc: 0.798306	valid_1's auc: 0.765495
[500]	training's auc: 0.805992	valid_1's auc: 0.768355
[600]	training's auc: 0.812846	valid_1's auc: 0.770316
[700]	training's auc: 0.819063	valid_1's auc: 0.771835
[800]	training's auc: 0.824847	valid_1's auc: 0.772954
[900]	training's auc: 0.830259	valid_1's auc: 0.773721
[1000]	training's auc: 0.8354	valid_1's auc: 0.774309
[1100]	training's auc: 0.840114	valid_1's auc: 0.774729
[1200]	training's auc: 0.844575	valid_1's auc: 0.774944
[1300]	training's auc: 0.848983	valid_1's auc: 0.775066
[1400]	training's auc: 0.853153	valid_1's auc: 0.775306
[1500]	training's auc: 0.857175	valid_1's 

In [40]:
imp.sort_values("imp", ascending=False)[:10]

,col,imp,imp_std
56,EXT_SOURCE_mean,321172.876397,18754.741575
182,NEW_EXT_SOURCES_GEO,165255.563773,13054.771304
184,NEW_EXT_SOURCES_MEDIAN,57047.194712,12382.094082
176,NEW_CREDIT_TO_ANNUITY_RATIO,51616.929113,1088.132902
10,ANNUITY_div_CREDIT,43046.548645,3830.491827
333,app_EXT_SOURCE_2_times_EXT_SOURCE_3,27987.619144,2839.886640
53,EXT_SOURCE_3,27681.110466,2606.986520
21,CNT_INSTALMENT_FUTURE_mean,26454.257148,991.878577
55,EXT_SOURCE_max,25917.402562,6346.193082
177,NEW_CREDIT_TO_GOODS_RATIO,23318.586936,926.782858


In [41]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [42]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [43]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_featureEngineering2.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.039484
1,100005,0.069016
2,100013,0.023776
3,100028,0.040394
4,100038,0.171242


In [44]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=="O":
        x_train[col] = x_train[col].astype('category')

In [45]:
#baseline = Baseline('SK_ID_CURR')

#sampler = optuna.samplers.TPESampler(seed=123)
#study = optuna.create_study(sampler=sampler, direction='maximize')
#study.optimize(objective, n_trials=50, n_jobs=5)

In [46]:
#trial = study.best_trial
#print("auc(best)={:.4f}".format(trial.value))
#display(trial.params)

In [47]:
#params_best = trial.params
#params_best.update(params_base)
#display(params_best)

In [48]:
#train_oof, imp, metrics = baseline.train_lgb(x_train
#                                    ,y_train
#                                    ,id_train
#                                    ,list_nfold=[0,1,2,3,4]
#                                    ,n_splits=5
#                                    ,params=params_best)

In [49]:
# 推論用データセットの作成
#x_test = df_test.drop(columns=['SK_ID_CURR'])
#id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
#for col in x_test.columns:
#    if x_test[col].dtype=="O":
#        x_test[col] = x_test[col].astype('category')

# predict
#test_pred = baseline.predict_lgb(x_test
#                       ,id_test
#                       ,list_nfold=[0,1,2,3,4])

# make submission-file
#df_submit = test_pred.rename(columns={"pred":"TARGET"})
#print(df_submit.shape)
#display(df_submit.head())
#df_submit.to_csv("./home-credit-default-risk/submit/submission_HyperParameterTuning.csv", index=None)

### ### credit card blanceの読み込み

In [50]:
credit = credit_card_balance_origin.copy()
cerdit = baseline.reduce_mem_usage(credit)
print(credit.shape)
credit.head()

Memory usage of dataframe is 673.88 MB
Memory usage of optimization is 334107272.00 MB
Decreased by -49579318.7%
(3840312, 23)


,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970001,135000,0.0,877.5,0.0,877.5,1700.324951,...,0.000000,0.000000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.554688,45000,2250.0,2250.0,0.0,0.0,2250.000000,...,64875.554688,64875.554688,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.224609,450000,0.0,0.0,0.0,0.0,2250.000000,...,31460.085938,31460.085938,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.109375,225000,2250.0,2250.0,0.0,0.0,11795.759766,...,233048.968750,233048.968750,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.468750,450000,0.0,11547.0,0.0,11547.0,22924.890625,...,453919.468750,453919.468750,0.0,1,0.0,1.0,101.0,Active,0,0


In [51]:
# one-hotencoding(処理対象カラムを取得)
encoding_columns = []

for col in credit.columns:
    if credit[col].dtype=="O":
        print(col)
        encoding_columns.append(col)

encoding_columns

NAME_CONTRACT_STATUS


['NAME_CONTRACT_STATUS']

In [52]:
# one-hot-encoding
for col in encoding_columns:
    credit_ohe = pd.get_dummies(credit, columns=[col], dummy_na=True)
    col_ohe = sorted(list(set(credit_ohe.columns) - set(credit.columns)))

print(len(col_ohe))
col_ohe

8


['NAME_CONTRACT_STATUS_Active',
 'NAME_CONTRACT_STATUS_Approved',
 'NAME_CONTRACT_STATUS_Completed',
 'NAME_CONTRACT_STATUS_Demand',
 'NAME_CONTRACT_STATUS_Refused',
 'NAME_CONTRACT_STATUS_Sent proposal',
 'NAME_CONTRACT_STATUS_Signed',
 'NAME_CONTRACT_STATUS_nan']

In [53]:
print(credit.columns)

Index(['SK_ID_PREV', 'SK_ID_CURR', 'MONTHS_BALANCE', 'AMT_BALANCE',
       'AMT_CREDIT_LIMIT_ACTUAL', 'AMT_DRAWINGS_ATM_CURRENT',
       'AMT_DRAWINGS_CURRENT', 'AMT_DRAWINGS_OTHER_CURRENT',
       'AMT_DRAWINGS_POS_CURRENT', 'AMT_INST_MIN_REGULARITY',
       'AMT_PAYMENT_CURRENT', 'AMT_PAYMENT_TOTAL_CURRENT',
       'AMT_RECEIVABLE_PRINCIPAL', 'AMT_RECIVABLE', 'AMT_TOTAL_RECEIVABLE',
       'CNT_DRAWINGS_ATM_CURRENT', 'CNT_DRAWINGS_CURRENT',
       'CNT_DRAWINGS_OTHER_CURRENT', 'CNT_DRAWINGS_POS_CURRENT',
       'CNT_INSTALMENT_MATURE_CUM', 'NAME_CONTRACT_STATUS', 'SK_DPD',
       'SK_DPD_DEF'],
      dtype='object')


In [54]:
credit_ohe_agg = credit_ohe.groupby('SK_ID_CURR').agg(
    {
        # 数値変数の集約
        'SK_ID_CURR': ['mean', 'std', 'min', 'max']
        ,'MONTHS_BALANCE': ['mean', 'std', 'min', 'max']
        ,'AMT_BALANCE': ['mean', 'std', 'min', 'max']
        ,'AMT_CREDIT_LIMIT_ACTUAL': ['mean', 'std', 'min', 'max']
        ,'AMT_DRAWINGS_ATM_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_DRAWINGS_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_DRAWINGS_OTHER_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_DRAWINGS_POS_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_INST_MIN_REGULARITY': ['mean', 'std', 'min', 'max']
        ,'AMT_PAYMENT_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_PAYMENT_TOTAL_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_RECEIVABLE_PRINCIPAL': ['mean', 'std', 'min', 'max']
        ,'AMT_RECIVABLE': ['mean', 'std', 'min', 'max']
        ,'AMT_TOTAL_RECEIVABLE': ['mean', 'std', 'min', 'max']
        ,'CNT_DRAWINGS_ATM_CURRENT': ['mean', 'std', 'min', 'max']
        ,'CNT_DRAWINGS_CURRENT': ['mean', 'std', 'min', 'max']
        ,'CNT_DRAWINGS_OTHER_CURRENT': ['mean', 'std', 'min', 'max']
        ,'CNT_DRAWINGS_POS_CURRENT': ['mean', 'std', 'min', 'max']
        ,'CNT_INSTALMENT_MATURE_CUM': ['mean', 'std', 'min', 'max']
        ,'SK_DPD': ['mean', 'std', 'min', 'max']
        ,'SK_DPD_DEF': ['mean', 'std', 'min', 'max']
        # 
        ,'NAME_CONTRACT_STATUS_Active': ['mean']
        ,'NAME_CONTRACT_STATUS_Approved': ['mean']
        ,'NAME_CONTRACT_STATUS_Completed': ['mean']
        ,'NAME_CONTRACT_STATUS_Demand': ['mean']
        ,'NAME_CONTRACT_STATUS_Refused': ['mean']
        ,'NAME_CONTRACT_STATUS_Sent proposal': ['mean']
        ,'NAME_CONTRACT_STATUS_Signed': ['mean']
        ,'NAME_CONTRACT_STATUS_nan': ['mean']
        # IDのユニーク数をカウント
        ,"SK_ID_PREV":["count", "nunique"]
    }
)

# カラム名の付与
credit_ohe_agg.columns = [i + "_" + j for i,j in credit_ohe_agg.columns]
credit_ohe_agg = credit_ohe_agg.reset_index(drop=False)

print(credit_ohe_agg.shape)
credit_ohe_agg.head()

(103558, 95)


,SK_ID_CURR,SK_ID_CURR_mean,SK_ID_CURR_std,SK_ID_CURR_min,SK_ID_CURR_max,MONTHS_BALANCE_mean,MONTHS_BALANCE_std,MONTHS_BALANCE_min,MONTHS_BALANCE_max,AMT_BALANCE_mean,...,NAME_CONTRACT_STATUS_Active_mean,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Refused_mean,NAME_CONTRACT_STATUS_Sent proposal_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100006,100006.0,0.0,100006,100006,-3.5,1.870829,-6,-1,0.000000,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,6,1
1,100011,100011.0,0.0,100011,100011,-38.5,21.505813,-75,-2,54482.109375,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,74,1
2,100013,100013.0,0.0,100013,100013,-48.5,27.856777,-96,-1,18159.919922,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,96,1
3,100021,100021.0,0.0,100021,100021,-10.0,5.049752,-18,-2,0.000000,...,0.411765,0.0,0.588235,0.0,0.0,0.0,0.0,0.0,17,1
4,100023,100023.0,0.0,100023,100023,-7.5,2.449490,-11,-4,0.000000,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,8,1


In [55]:
# infをnanへ置換
credit_ohe_agg = credit_ohe_agg.replace([np.inf, -np.inf], np.nan)

In [56]:
df_train = pd.merge(app_train, credit_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, pos_ohe_agg, on='SK_ID_CURR', how='left')
print(df_train.shape)
df_train.head()

(307506, 440)


,level_0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,NAME_CONTRACT_STATUS_Approved_mean_y,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean_y,NAME_CONTRACT_STATUS_Demand_mean_y,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean_y,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean_y,SK_ID_PREV_count_y,SK_ID_PREV_nunique_y
0,0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,19.0,1.0
1,1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,...,0.0,0.0,0.071429,0.0,0.000000,0.000000,0.0,0.0,28.0,3.0
2,2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,...,0.0,0.0,0.250000,0.0,0.000000,0.000000,0.0,0.0,4.0,1.0
3,3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,...,0.0,0.0,0.095238,0.0,0.047619,0.000000,0.0,0.0,21.0,3.0
4,4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,...,0.0,0.0,0.045455,0.0,0.000000,0.015152,0.0,0.0,66.0,5.0


In [57]:
# テーブル結合
df_test = pd.merge(app_test, credit_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, pos_ohe_agg, on='SK_ID_CURR', how='left')
print(df_test.shape)
df_test.head()

(48744, 439)


,level_0,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Approved_mean_y,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean_y,NAME_CONTRACT_STATUS_Demand_mean_y,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean_y,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean_y,SK_ID_PREV_count_y,SK_ID_PREV_nunique_y
0,307511,100001,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,...,0.0,0.0,0.222222,0.0,0.0,0.000000,0.0,0.0,9.0,2.0
1,307512,100005,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,...,0.0,0.0,0.090909,0.0,0.0,0.090909,0.0,0.0,11.0,1.0
2,307513,100013,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,...,0.0,0.0,0.083333,0.0,0.0,0.027778,0.0,0.0,36.0,3.0
3,307514,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,...,0.0,0.0,0.064516,0.0,0.0,0.000000,0.0,0.0,31.0,2.0
4,307515,100038,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,...,0.0,0.0,0.076923,0.0,0.0,0.000000,0.0,0.0,13.0,1.0


In [58]:
### 処理対象の列名を配列化
#encoding_columns = []
encoding_columns = list(df_test.select_dtypes(include='number').columns)

#for col in df_test.columns:
#    print(col)
#    encoding_columns.append(col)
    
encoding_columns.remove('SK_ID_CURR')

In [59]:
### Yao-Jhonson変換
for column in encoding_columns:
    modules.process_yj_convert(df_train, df_test, column, [column])

df_train.head()

,level_0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,NAME_CONTRACT_STATUS_Approved_mean_y,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean_y,NAME_CONTRACT_STATUS_Demand_mean_y,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean_y,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean_y,SK_ID_PREV_count_y,SK_ID_PREV_nunique_y
0,-2.196836,100002,1.0,-0.653131,0.647085,-0.284046,0.026932,-0.324380,0.047085,1.597501,...,-0.117324,-0.006439,-1.882714,-0.03285,-0.122221,-0.493724,-0.002629,0.0,-0.187757,-1.255497
1,-2.196523,100003,0.0,-0.653131,1.213324,1.495396,0.744618,1.433694,-1.730864,-0.222757,...,-0.117324,-0.006439,-0.045923,-0.03285,-0.122221,-0.493724,-0.002629,0.0,0.304656,0.504269
2,-2.196256,100004,0.0,-0.653131,-1.654456,-1.648319,-2.082107,-1.533668,-0.863026,-0.716549,...,-0.117324,-0.006439,2.215187,-0.03285,-0.122221,-0.493724,-0.002629,0.0,-2.038472,-1.255497
3,-2.196014,100006,0.0,-0.653131,-0.176134,-0.635707,0.377928,-0.549543,-1.112439,-0.707904,...,-0.117324,-0.006439,0.405962,-0.03285,8.181937,-0.493724,-0.002629,0.0,-0.061302,0.504269
4,-2.195789,100007,0.0,-0.653131,-0.394983,0.042350,-0.198042,0.210360,0.799714,-0.901463,...,-0.117324,-0.006439,-0.620243,-0.03285,-0.122221,1.480653,-0.002629,0.0,1.408833,1.246159


In [60]:
### indexを設定
df_train = df_train.set_index('SK_ID_CURR')
df_test = df_test.set_index('SK_ID_CURR')
df_train.head()

,level_0,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Approved_mean_y,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean_y,NAME_CONTRACT_STATUS_Demand_mean_y,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean_y,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean_y,SK_ID_PREV_count_y,SK_ID_PREV_nunique_y
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,-2.196836,1.0,-0.653131,0.647085,-0.284046,0.026932,-0.324380,0.047085,1.597501,0.846146,...,-0.117324,-0.006439,-1.882714,-0.03285,-0.122221,-0.493724,-0.002629,0.0,-0.187757,-1.255497
100003,-2.196523,0.0,-0.653131,1.213324,1.495396,0.744618,1.433694,-1.730864,-0.222757,0.295507,...,-0.117324,-0.006439,-0.045923,-0.03285,-0.122221,-0.493724,-0.002629,0.0,0.304656,0.504269
100004,-2.196256,0.0,-0.653131,-1.654456,-1.648319,-2.082107,-1.533668,-0.863026,-0.716549,1.649642,...,-0.117324,-0.006439,2.215187,-0.03285,-0.122221,-0.493724,-0.002629,0.0,-2.038472,-1.255497
100006,-2.196014,0.0,-0.653131,-0.176134,-0.635707,0.377928,-0.549543,-1.112439,-0.707904,-0.647130,...,-0.117324,-0.006439,0.405962,-0.03285,8.181937,-0.493724,-0.002629,0.0,-0.061302,0.504269
100007,-2.195789,0.0,-0.653131,-0.394983,0.042350,-0.198042,0.210360,0.799714,-0.901463,-0.646418,...,-0.117324,-0.006439,-0.620243,-0.03285,-0.122221,1.480653,-0.002629,0.0,1.408833,1.246159


In [61]:
### 標準化
for column in encoding_columns:
    num_cols = []
    num_cols.append(column)
    result = modules.process_standardization(df_train, df_test, num_cols)
        
df_train.head()

,level_0,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Approved_mean_y,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean_y,NAME_CONTRACT_STATUS_Demand_mean_y,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean_y,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean_y,SK_ID_PREV_count_y,SK_ID_PREV_nunique_y
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,-2.196836,1.0,-0.653131,0.647085,-0.284046,0.026932,-0.324380,0.047085,1.597501,0.846146,...,-0.117324,-0.006439,-1.882714,-0.03285,-0.122221,-0.493724,-0.002629,0.0,-0.187757,-1.255497
100003,-2.196523,0.0,-0.653131,1.213324,1.495396,0.744618,1.433694,-1.730864,-0.222757,0.295507,...,-0.117324,-0.006439,-0.045923,-0.03285,-0.122221,-0.493724,-0.002629,0.0,0.304656,0.504269
100004,-2.196256,0.0,-0.653131,-1.654456,-1.648319,-2.082107,-1.533668,-0.863026,-0.716549,1.649642,...,-0.117324,-0.006439,2.215187,-0.03285,-0.122221,-0.493724,-0.002629,0.0,-2.038472,-1.255497
100006,-2.196014,0.0,-0.653131,-0.176134,-0.635707,0.377928,-0.549543,-1.112439,-0.707904,-0.647130,...,-0.117324,-0.006439,0.405962,-0.03285,8.181937,-0.493724,-0.002629,0.0,-0.061302,0.504269
100007,-2.195789,0.0,-0.653131,-0.394983,0.042350,-0.198042,0.210360,0.799714,-0.901463,-0.646418,...,-0.117324,-0.006439,-0.620243,-0.03285,-0.122221,1.480653,-0.002629,0.0,1.408833,1.246159


In [62]:
### indexを削除
df_train = df_train.reset_index()
df_test = df_test.reset_index()
df_train.head()

,SK_ID_CURR,level_0,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,NAME_CONTRACT_STATUS_Approved_mean_y,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean_y,NAME_CONTRACT_STATUS_Demand_mean_y,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean_y,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean_y,SK_ID_PREV_count_y,SK_ID_PREV_nunique_y
0,100002,-2.196836,1.0,-0.653131,0.647085,-0.284046,0.026932,-0.324380,0.047085,1.597501,...,-0.117324,-0.006439,-1.882714,-0.03285,-0.122221,-0.493724,-0.002629,0.0,-0.187757,-1.255497
1,100003,-2.196523,0.0,-0.653131,1.213324,1.495396,0.744618,1.433694,-1.730864,-0.222757,...,-0.117324,-0.006439,-0.045923,-0.03285,-0.122221,-0.493724,-0.002629,0.0,0.304656,0.504269
2,100004,-2.196256,0.0,-0.653131,-1.654456,-1.648319,-2.082107,-1.533668,-0.863026,-0.716549,...,-0.117324,-0.006439,2.215187,-0.03285,-0.122221,-0.493724,-0.002629,0.0,-2.038472,-1.255497
3,100006,-2.196014,0.0,-0.653131,-0.176134,-0.635707,0.377928,-0.549543,-1.112439,-0.707904,...,-0.117324,-0.006439,0.405962,-0.03285,8.181937,-0.493724,-0.002629,0.0,-0.061302,0.504269
4,100007,-2.195789,0.0,-0.653131,-0.394983,0.042350,-0.198042,0.210360,0.799714,-0.901463,...,-0.117324,-0.006439,-0.620243,-0.03285,-0.122221,1.480653,-0.002629,0.0,1.408833,1.246159


In [63]:
### メモリサイズを縮小
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()

Memory usage of dataframe is 655.73 MB
Memory usage of optimization is 541210688.00 MB
Decreased by -82535403.3%
Memory usage of dataframe is 103.76 MB
Memory usage of optimization is 85594592.00 MB
Decreased by -82495421.6%


,SK_ID_CURR,level_0,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,NAME_CONTRACT_STATUS_Approved_mean_y,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean_y,NAME_CONTRACT_STATUS_Demand_mean_y,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean_y,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean_y,SK_ID_PREV_count_y,SK_ID_PREV_nunique_y
0,100002,-2.197266,1.0,-0.65332,0.646973,-0.283936,0.026932,-0.324463,0.047089,1.597656,...,-0.11731,-0.006439,-1.882812,-0.032837,-0.122192,-0.493652,-0.002628,0.0,-0.187744,-1.255859
1,100003,-2.197266,0.0,-0.65332,1.212891,1.495117,0.744629,1.433594,-1.730469,-0.222778,...,-0.11731,-0.006439,-0.045929,-0.032837,-0.122192,-0.493652,-0.002628,0.0,0.304688,0.504395
2,100004,-2.195312,0.0,-0.65332,-1.654297,-1.648438,-2.082031,-1.533203,-0.862793,-0.716309,...,-0.11731,-0.006439,2.214844,-0.032837,-0.122192,-0.493652,-0.002628,0.0,-2.039062,-1.255859
3,100006,-2.195312,0.0,-0.65332,-0.176147,-0.635742,0.377930,-0.549316,-1.112305,-0.708008,...,-0.11731,-0.006439,0.406006,-0.032837,8.179688,-0.493652,-0.002628,0.0,-0.061310,0.504395
4,100007,-2.195312,0.0,-0.65332,-0.395020,0.042358,-0.197998,0.210327,0.799805,-0.901367,...,-0.11731,-0.006439,-0.620117,-0.032837,-0.122192,1.480469,-0.002628,0.0,1.409180,1.246094


In [64]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=="O":
        x_train[col] = x_train[col].astype('category')

In [65]:
baseline = Baseline('SK_ID_CURR')

"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.05
    ,'num_leaves': 32
    ,'n_estimators': 100000
    ,'random_state':123
    ,'importance_type': 'gain'
}
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    #,'device_type': 'gpu'
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'num_leaves': 58
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_weight': 6
    ,'min_child_samples': 165
    #,'keep_training_booster': True
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}

train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246004, 438) (61502, 438)
[LightGBM] [Warning] num_threads is set with n_jobs=-1, nthread=-1 will be ignored. Current value: num_threads=-1
[100]	training's auc: 0.767812	valid_1's auc: 0.748247
[200]	training's auc: 0.780974	valid_1's auc: 0.756657
[300]	training's auc: 0.791902	valid_1's auc: 0.762252
[400]	training's auc: 0.801195	valid_1's auc: 0.766993
[500]	training's auc: 0.808906	valid_1's auc: 0.770034
[600]	training's auc: 0.815631	valid_1's auc: 0.771957
[700]	training's auc: 0.821832	valid_1's auc: 0.773542
[800]	training's auc: 0.827625	valid_1's auc: 0.774697
[900]	training's auc: 0.833005	valid_1's auc: 0.775492
[1000]	training's auc: 0.838091	valid_1's auc: 0.77615
[1100]	training's auc: 0.84284	valid_1's auc: 0.776602
[1200]	training's auc: 0.847428	valid_1's auc: 0.777
[1300]	training's auc: 0.85169	valid_1's auc: 0.777262
[1400]	training's auc: 0.855762	valid_1's auc: 0.777447
[1500]	training's auc: 0.859644	valid_1's auc:

In [66]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
124,EXT_SOURCE_mean,302329.710254,24999.089244
262,NEW_EXT_SOURCES_GEO,163874.275361,10040.188080
264,NEW_EXT_SOURCES_MEDIAN,66567.102078,13029.996395
256,NEW_CREDIT_TO_ANNUITY_RATIO,52214.180928,2804.288391
58,ANNUITY_div_CREDIT,38007.804392,1901.739965
427,app_EXT_SOURCE_2_times_EXT_SOURCE_3,28801.845195,2899.728646
123,EXT_SOURCE_max,27497.042486,3719.016358
121,EXT_SOURCE_3,26610.854641,2436.383451
85,CNT_INSTALMENT_FUTURE_mean,25392.509985,1036.462194
195,MONTHS_BALANCE_std_y,23226.247990,1366.964889


In [67]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [68]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [69]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_featureEngineering3.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.039553
1,100005,0.068104
2,100013,0.027129
3,100028,0.046693
4,100038,0.170886


### previous applicationの読み込み

In [70]:
previous = previous_application_origin.copy()
previous = baseline.reduce_mem_usage(credit)
print(previous.shape)
previous.head()

Memory usage of dataframe is 318.63 MB
Memory usage of optimization is 334107272.00 MB
Decreased by -104857500.0%
(3840312, 23)


,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970001,135000,0.0,877.5,0.0,877.5,1700.324951,...,0.000000,0.000000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.554688,45000,2250.0,2250.0,0.0,0.0,2250.000000,...,64875.554688,64875.554688,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.224609,450000,0.0,0.0,0.0,0.0,2250.000000,...,31460.085938,31460.085938,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.109375,225000,2250.0,2250.0,0.0,0.0,11795.759766,...,233048.968750,233048.968750,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.468750,450000,0.0,11547.0,0.0,11547.0,22924.890625,...,453919.468750,453919.468750,0.0,1,0.0,1.0,101.0,Active,0,0


In [71]:
# one-hotencoding(処理対象カラムを取得)
encoding_columns = []

for col in previous.columns:
    if previous[col].dtype=="O":
        print(col)
        encoding_columns.append(col)

#encoding_columns

NAME_CONTRACT_STATUS


In [72]:
# one-hot-encoding
for col in encoding_columns:
    previous_ohe = pd.get_dummies(previous, columns=[col], dummy_na=True)
    col_ohe = sorted(list(set(previous_ohe.columns) - set(previous.columns)))

print(len(col_ohe))
col_ohe

8


['NAME_CONTRACT_STATUS_Active',
 'NAME_CONTRACT_STATUS_Approved',
 'NAME_CONTRACT_STATUS_Completed',
 'NAME_CONTRACT_STATUS_Demand',
 'NAME_CONTRACT_STATUS_Refused',
 'NAME_CONTRACT_STATUS_Sent proposal',
 'NAME_CONTRACT_STATUS_Signed',
 'NAME_CONTRACT_STATUS_nan']

In [73]:
print(previous.columns)

Index(['SK_ID_PREV', 'SK_ID_CURR', 'MONTHS_BALANCE', 'AMT_BALANCE',
       'AMT_CREDIT_LIMIT_ACTUAL', 'AMT_DRAWINGS_ATM_CURRENT',
       'AMT_DRAWINGS_CURRENT', 'AMT_DRAWINGS_OTHER_CURRENT',
       'AMT_DRAWINGS_POS_CURRENT', 'AMT_INST_MIN_REGULARITY',
       'AMT_PAYMENT_CURRENT', 'AMT_PAYMENT_TOTAL_CURRENT',
       'AMT_RECEIVABLE_PRINCIPAL', 'AMT_RECIVABLE', 'AMT_TOTAL_RECEIVABLE',
       'CNT_DRAWINGS_ATM_CURRENT', 'CNT_DRAWINGS_CURRENT',
       'CNT_DRAWINGS_OTHER_CURRENT', 'CNT_DRAWINGS_POS_CURRENT',
       'CNT_INSTALMENT_MATURE_CUM', 'NAME_CONTRACT_STATUS', 'SK_DPD',
       'SK_DPD_DEF'],
      dtype='object')


In [74]:
previous_ohe_agg = credit_ohe.groupby('SK_ID_CURR').agg(
    {
        # 数値変数の集約
        'MONTHS_BALANCE': ['mean', 'std', 'min', 'max']
        ,'AMT_BALANCE': ['mean', 'std', 'min', 'max']
        ,'AMT_CREDIT_LIMIT_ACTUAL': ['mean', 'std', 'min', 'max']
        ,'AMT_DRAWINGS_ATM_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_DRAWINGS_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_DRAWINGS_OTHER_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_DRAWINGS_POS_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_INST_MIN_REGULARITY': ['mean', 'std', 'min', 'max']
        ,'AMT_PAYMENT_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_PAYMENT_TOTAL_CURRENT': ['mean', 'std', 'min', 'max']
        ,'AMT_RECEIVABLE_PRINCIPAL': ['mean', 'std', 'min', 'max']
        ,'AMT_RECIVABLE': ['mean', 'std', 'min', 'max']
        ,'AMT_TOTAL_RECEIVABLE': ['mean', 'std', 'min', 'max']
        ,'CNT_DRAWINGS_ATM_CURRENT': ['mean', 'std', 'min', 'max']
        ,'CNT_DRAWINGS_CURRENT': ['mean', 'std', 'min', 'max']
        ,'CNT_DRAWINGS_OTHER_CURRENT': ['mean', 'std', 'min', 'max']
        ,'CNT_DRAWINGS_POS_CURRENT': ['mean', 'std', 'min', 'max']
        ,'CNT_INSTALMENT_MATURE_CUM': ['mean', 'std', 'min', 'max']
        ,'SK_DPD': ['mean', 'std', 'min', 'max']
        ,'SK_DPD_DEF': ['mean', 'std', 'min', 'max']
        # 
        ,'NAME_CONTRACT_STATUS_Active': ['mean']
        ,'NAME_CONTRACT_STATUS_Approved': ['mean']
        ,'NAME_CONTRACT_STATUS_Completed': ['mean']
        ,'NAME_CONTRACT_STATUS_Demand': ['mean']
        ,'NAME_CONTRACT_STATUS_Refused': ['mean']
        ,'NAME_CONTRACT_STATUS_Sent proposal': ['mean']
        ,'NAME_CONTRACT_STATUS_Signed': ['mean']
        ,'NAME_CONTRACT_STATUS_nan': ['mean']
        # IDのユニーク数をカウント
        ,"SK_ID_PREV":["count", "nunique"]
    }
)

# カラム名の付与
previous_ohe_agg.columns = [i + "_" + j for i,j in previous_ohe_agg.columns]
previous_ohe_agg = previous_ohe_agg.reset_index(drop=False)

print(previous_ohe_agg.shape)
previous_ohe_agg.head()

(103558, 91)


,SK_ID_CURR,MONTHS_BALANCE_mean,MONTHS_BALANCE_std,MONTHS_BALANCE_min,MONTHS_BALANCE_max,AMT_BALANCE_mean,AMT_BALANCE_std,AMT_BALANCE_min,AMT_BALANCE_max,AMT_CREDIT_LIMIT_ACTUAL_mean,...,NAME_CONTRACT_STATUS_Active_mean,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Refused_mean,NAME_CONTRACT_STATUS_Sent proposal_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100006,-3.5,1.870829,-6,-1,0.000000,0.000000,0.0,0.00000,270000.000000,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,6,1
1,100011,-38.5,21.505813,-75,-2,54482.109375,68127.238133,0.0,189000.00000,164189.189189,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,74,1
2,100013,-48.5,27.856777,-96,-1,18159.919922,43237.406465,0.0,161420.21875,131718.750000,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,96,1
3,100021,-10.0,5.049752,-18,-2,0.000000,0.000000,0.0,0.00000,675000.000000,...,0.411765,0.0,0.588235,0.0,0.0,0.0,0.0,0.0,17,1
4,100023,-7.5,2.449490,-11,-4,0.000000,0.000000,0.0,0.00000,135000.000000,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,8,1


In [75]:
# infをnanへ置換
previous_ohe_agg = previous_ohe_agg.replace([np.inf, -np.inf], np.nan)

In [76]:
df_train = pd.merge(app_train, previous_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, credit_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, pos_ohe_agg, on='SK_ID_CURR', how='left')
print(df_train.shape)
df_train.head()

(307506, 530)


,level_0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,19.0,1.0
1,1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,...,0.0,0.0,0.071429,0.0,0.000000,0.000000,0.0,0.0,28.0,3.0
2,2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,...,0.0,0.0,0.250000,0.0,0.000000,0.000000,0.0,0.0,4.0,1.0
3,3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,...,0.0,0.0,0.095238,0.0,0.047619,0.000000,0.0,0.0,21.0,3.0
4,4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,...,0.0,0.0,0.045455,0.0,0.000000,0.015152,0.0,0.0,66.0,5.0


In [77]:
# テーブル結合
df_test = pd.merge(app_test, previous_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, credit_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, pos_ohe_agg, on='SK_ID_CURR', how='left')
print(df_test.shape)
df_test.head()

(48744, 529)


,level_0,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,307511,100001,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,...,0.0,0.0,0.222222,0.0,0.0,0.000000,0.0,0.0,9.0,2.0
1,307512,100005,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,...,0.0,0.0,0.090909,0.0,0.0,0.090909,0.0,0.0,11.0,1.0
2,307513,100013,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,...,0.0,0.0,0.083333,0.0,0.0,0.027778,0.0,0.0,36.0,3.0
3,307514,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,...,0.0,0.0,0.064516,0.0,0.0,0.000000,0.0,0.0,31.0,2.0
4,307515,100038,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,...,0.0,0.0,0.076923,0.0,0.0,0.000000,0.0,0.0,13.0,1.0


In [78]:
### 処理対象の列名を配列化
#encoding_columns = []
encoding_columns = list(df_test.select_dtypes(include='number').columns)

#for col in df_test.columns:
#    print(col)
#    encoding_columns.append(col)
    
encoding_columns.remove('SK_ID_CURR')

In [79]:
### Yao-Jhonson変換
for column in encoding_columns:
    modules.process_yj_convert(df_train, df_test, column, [column])

df_train.head()

,level_0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,-2.196836,100002,1.0,-0.653131,0.647085,-0.284046,0.026932,-0.324380,0.047085,1.597501,...,-0.117324,-0.006439,-1.882714,-0.03285,-0.122221,-0.493724,-0.002629,0.0,-0.187757,-1.255497
1,-2.196523,100003,0.0,-0.653131,1.213324,1.495396,0.744618,1.433694,-1.730864,-0.222757,...,-0.117324,-0.006439,-0.045923,-0.03285,-0.122221,-0.493724,-0.002629,0.0,0.304656,0.504269
2,-2.196256,100004,0.0,-0.653131,-1.654456,-1.648319,-2.082107,-1.533668,-0.863026,-0.716549,...,-0.117324,-0.006439,2.215187,-0.03285,-0.122221,-0.493724,-0.002629,0.0,-2.038472,-1.255497
3,-2.196014,100006,0.0,-0.653131,-0.176134,-0.635707,0.377928,-0.549543,-1.112439,-0.707904,...,-0.117324,-0.006439,0.405962,-0.03285,8.181937,-0.493724,-0.002629,0.0,-0.061302,0.504269
4,-2.195789,100007,0.0,-0.653131,-0.394983,0.042350,-0.198042,0.210360,0.799714,-0.901463,...,-0.117324,-0.006439,-0.620243,-0.03285,-0.122221,1.480653,-0.002629,0.0,1.408833,1.246159


In [80]:
### indexを設定
df_train = df_train.set_index('SK_ID_CURR')
df_test = df_test.set_index('SK_ID_CURR')
df_train.head()

,level_0,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,-2.196836,1.0,-0.653131,0.647085,-0.284046,0.026932,-0.324380,0.047085,1.597501,0.846146,...,-0.117324,-0.006439,-1.882714,-0.03285,-0.122221,-0.493724,-0.002629,0.0,-0.187757,-1.255497
100003,-2.196523,0.0,-0.653131,1.213324,1.495396,0.744618,1.433694,-1.730864,-0.222757,0.295507,...,-0.117324,-0.006439,-0.045923,-0.03285,-0.122221,-0.493724,-0.002629,0.0,0.304656,0.504269
100004,-2.196256,0.0,-0.653131,-1.654456,-1.648319,-2.082107,-1.533668,-0.863026,-0.716549,1.649642,...,-0.117324,-0.006439,2.215187,-0.03285,-0.122221,-0.493724,-0.002629,0.0,-2.038472,-1.255497
100006,-2.196014,0.0,-0.653131,-0.176134,-0.635707,0.377928,-0.549543,-1.112439,-0.707904,-0.647130,...,-0.117324,-0.006439,0.405962,-0.03285,8.181937,-0.493724,-0.002629,0.0,-0.061302,0.504269
100007,-2.195789,0.0,-0.653131,-0.394983,0.042350,-0.198042,0.210360,0.799714,-0.901463,-0.646418,...,-0.117324,-0.006439,-0.620243,-0.03285,-0.122221,1.480653,-0.002629,0.0,1.408833,1.246159


In [81]:
### 標準化
for column in encoding_columns:
    num_cols = []
    num_cols.append(column)
    result = modules.process_standardization(df_train, df_test, num_cols)
        
df_train.head()

,level_0,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,-2.196836,1.0,-0.653131,0.647085,-0.284046,0.026932,-0.324380,0.047085,1.597501,0.846146,...,-0.117324,-0.006439,-1.882714,-0.03285,-0.122221,-0.493724,-0.002629,0.0,-0.187757,-1.255497
100003,-2.196523,0.0,-0.653131,1.213324,1.495396,0.744618,1.433694,-1.730864,-0.222757,0.295507,...,-0.117324,-0.006439,-0.045923,-0.03285,-0.122221,-0.493724,-0.002629,0.0,0.304656,0.504269
100004,-2.196256,0.0,-0.653131,-1.654456,-1.648319,-2.082107,-1.533668,-0.863026,-0.716549,1.649642,...,-0.117324,-0.006439,2.215187,-0.03285,-0.122221,-0.493724,-0.002629,0.0,-2.038472,-1.255497
100006,-2.196014,0.0,-0.653131,-0.176134,-0.635707,0.377928,-0.549543,-1.112439,-0.707904,-0.647130,...,-0.117324,-0.006439,0.405962,-0.03285,8.181937,-0.493724,-0.002629,0.0,-0.061302,0.504269
100007,-2.195789,0.0,-0.653131,-0.394983,0.042350,-0.198042,0.210360,0.799714,-0.901463,-0.646418,...,-0.117324,-0.006439,-0.620243,-0.03285,-0.122221,1.480653,-0.002629,0.0,1.408833,1.246159


In [82]:
### indexを削除
df_train = df_train.reset_index()
df_test = df_test.reset_index()
df_train.head()

,SK_ID_CURR,level_0,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100002,-2.196836,1.0,-0.653131,0.647085,-0.284046,0.026932,-0.324380,0.047085,1.597501,...,-0.117324,-0.006439,-1.882714,-0.03285,-0.122221,-0.493724,-0.002629,0.0,-0.187757,-1.255497
1,100003,-2.196523,0.0,-0.653131,1.213324,1.495396,0.744618,1.433694,-1.730864,-0.222757,...,-0.117324,-0.006439,-0.045923,-0.03285,-0.122221,-0.493724,-0.002629,0.0,0.304656,0.504269
2,100004,-2.196256,0.0,-0.653131,-1.654456,-1.648319,-2.082107,-1.533668,-0.863026,-0.716549,...,-0.117324,-0.006439,2.215187,-0.03285,-0.122221,-0.493724,-0.002629,0.0,-2.038472,-1.255497
3,100006,-2.196014,0.0,-0.653131,-0.176134,-0.635707,0.377928,-0.549543,-1.112439,-0.707904,...,-0.117324,-0.006439,0.405962,-0.03285,8.181937,-0.493724,-0.002629,0.0,-0.061302,0.504269
4,100007,-2.195789,0.0,-0.653131,-0.394983,0.042350,-0.198042,0.210360,0.799714,-0.901463,...,-0.117324,-0.006439,-0.620243,-0.03285,-0.122221,1.480653,-0.002629,0.0,1.408833,1.246159


In [83]:
### メモリサイズを縮小
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()

Memory usage of dataframe is 814.09 MB
Memory usage of optimization is 651912848.00 MB
Decreased by -80078474.3%
Memory usage of dataframe is 128.86 MB
Memory usage of optimization is 103142432.00 MB
Decreased by -80042737.9%


,SK_ID_CURR,level_0,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100002,-2.197266,1.0,-0.65332,0.646973,-0.283936,0.026932,-0.324463,0.047089,1.597656,...,-0.11731,-0.006439,-1.882812,-0.032837,-0.122192,-0.493652,-0.002628,0.0,-0.187744,-1.255859
1,100003,-2.197266,0.0,-0.65332,1.212891,1.495117,0.744629,1.433594,-1.730469,-0.222778,...,-0.11731,-0.006439,-0.045929,-0.032837,-0.122192,-0.493652,-0.002628,0.0,0.304688,0.504395
2,100004,-2.195312,0.0,-0.65332,-1.654297,-1.648438,-2.082031,-1.533203,-0.862793,-0.716309,...,-0.11731,-0.006439,2.214844,-0.032837,-0.122192,-0.493652,-0.002628,0.0,-2.039062,-1.255859
3,100006,-2.195312,0.0,-0.65332,-0.176147,-0.635742,0.377930,-0.549316,-1.112305,-0.708008,...,-0.11731,-0.006439,0.406006,-0.032837,8.179688,-0.493652,-0.002628,0.0,-0.061310,0.504395
4,100007,-2.195312,0.0,-0.65332,-0.395020,0.042358,-0.197998,0.210327,0.799805,-0.901367,...,-0.11731,-0.006439,-0.620117,-0.032837,-0.122192,1.480469,-0.002628,0.0,1.409180,1.246094


In [84]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=="O":
        x_train[col] = x_train[col].astype('category')

In [85]:
baseline = Baseline('SK_ID_CURR')

train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246004, 528) (61502, 528)
[LightGBM] [Warning] num_threads is set with n_jobs=-1, nthread=-1 will be ignored. Current value: num_threads=-1
[100]	training's auc: 0.767611	valid_1's auc: 0.748051
[200]	training's auc: 0.781146	valid_1's auc: 0.756601
[300]	training's auc: 0.792022	valid_1's auc: 0.762348
[400]	training's auc: 0.801394	valid_1's auc: 0.766935
[500]	training's auc: 0.809087	valid_1's auc: 0.769982
[600]	training's auc: 0.815865	valid_1's auc: 0.772052
[700]	training's auc: 0.822185	valid_1's auc: 0.773599
[800]	training's auc: 0.827891	valid_1's auc: 0.774833
[900]	training's auc: 0.833316	valid_1's auc: 0.775691
[1000]	training's auc: 0.838324	valid_1's auc: 0.77622
[1100]	training's auc: 0.843035	valid_1's auc: 0.776732
[1200]	training's auc: 0.847541	valid_1's auc: 0.777033
[1300]	training's auc: 0.851836	valid_1's auc: 0.777358
[1400]	training's auc: 0.855962	valid_1's auc: 0.777629
[1500]	training's auc: 0.859905	valid_1's

In [86]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
192,EXT_SOURCE_mean,325636.414168,11253.164399
342,NEW_EXT_SOURCES_GEO,160159.741878,7484.180003
336,NEW_CREDIT_TO_ANNUITY_RATIO,52886.167117,2128.657759
344,NEW_EXT_SOURCES_MEDIAN,48860.940269,2551.981668
106,ANNUITY_div_CREDIT,35483.728252,1843.949923
517,app_EXT_SOURCE_2_times_EXT_SOURCE_3,28882.490854,2641.913651
191,EXT_SOURCE_max,28871.213062,2794.958448
189,EXT_SOURCE_3,26051.998574,2230.007705
149,CNT_INSTALMENT_FUTURE_mean,24803.383864,813.154414
193,EXT_SOURCE_min,22823.868671,7024.934369


In [87]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [88]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [89]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_featureEngineering4.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.039883
1,100005,0.072499
2,100013,0.026741
3,100028,0.043844
4,100038,0.171433


### installments paymentsの読み込み

In [90]:
installments = installments_payments_origin.copy()
installments = baseline.reduce_mem_usage(installments)
print(installments.shape)
installments.head()

Memory usage of dataframe is 830.41 MB
Memory usage of optimization is 408162158.00 MB
Decreased by -49151908.2%
(13605401, 8)


,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.359863,6948.359863
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525024,1716.525024
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000000,25425.000000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130859,24350.130859
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040039,2160.584961


In [91]:
# one-hotencoding(処理対象カラムを取得)
encoding_columns = []

for col in installments.columns:
    if installments[col].dtype=="O":
        print(col)
        encoding_columns.append(col)

encoding_columns

[]

In [92]:
# one-hot-encoding
#for col in encoding_columns:
#    installments_ohe = pd.get_dummies(installments, columns=[col], dummy_na=True)
#    col_ohe = sorted(list(set(installments_ohe.columns) - set(installments.columns)))

#print(len(col_ohe))
#col_ohe
installments_ohe = installments.copy()

In [93]:
print(installments.columns)

Index(['SK_ID_PREV', 'SK_ID_CURR', 'NUM_INSTALMENT_VERSION',
       'NUM_INSTALMENT_NUMBER', 'DAYS_INSTALMENT', 'DAYS_ENTRY_PAYMENT',
       'AMT_INSTALMENT', 'AMT_PAYMENT'],
      dtype='object')


In [94]:
installments_ohe = installments_ohe.rename(columns={'SK_ID_PREV': 'SK_ID_PREV_IP'})

In [95]:
installments_ohe_agg = installments_ohe.groupby('SK_ID_CURR').agg(
    {
        # 数値変数の集約
        'NUM_INSTALMENT_VERSION': ['mean', 'std', 'min', 'max']
        ,'NUM_INSTALMENT_NUMBER': ['mean', 'std', 'min', 'max']
        ,'DAYS_INSTALMENT': ['mean', 'std', 'min', 'max']
        ,'DAYS_ENTRY_PAYMENT': ['mean', 'std', 'min', 'max']
        ,'AMT_INSTALMENT': ['mean', 'std', 'min', 'max']
        ,'AMT_PAYMENT': ['mean', 'std', 'min', 'max']
        # IDのユニーク数をカウント
        ,"SK_ID_PREV_IP":["count", "nunique"]
    }
)

# カラム名の付与
installments_ohe_agg.columns = [i + "_" + j for i,j in installments_ohe_agg.columns]
installments_ohe_agg = installments_ohe_agg.reset_index(drop=False)

print(installments_ohe_agg.shape)
installments_ohe_agg.head()

(339587, 27)


,SK_ID_CURR,NUM_INSTALMENT_VERSION_mean,NUM_INSTALMENT_VERSION_std,NUM_INSTALMENT_VERSION_min,NUM_INSTALMENT_VERSION_max,NUM_INSTALMENT_NUMBER_mean,NUM_INSTALMENT_NUMBER_std,NUM_INSTALMENT_NUMBER_min,NUM_INSTALMENT_NUMBER_max,DAYS_INSTALMENT_mean,...,AMT_INSTALMENT_mean,AMT_INSTALMENT_std,AMT_INSTALMENT_min,AMT_INSTALMENT_max,AMT_PAYMENT_mean,AMT_PAYMENT_std,AMT_PAYMENT_min,AMT_PAYMENT_max,SK_ID_PREV_IP_count,SK_ID_PREV_IP_nunique
0,100001,1.142857,0.377964,1.0,2.0,2.714286,1.112697,1,4,-2187.714355,...,5885.132324,5076.676763,3951.000000,17397.900391,5885.132324,5076.676763,3951.000000,17397.900391,7,2
1,100002,1.052632,0.229416,1.0,2.0,10.000000,5.627314,1,19,-295.000000,...,11559.247070,10058.037883,9251.775391,53093.746094,11559.247070,10058.037883,9251.775391,53093.746094,19,1
2,100003,1.040000,0.200000,1.0,2.0,5.080000,3.134751,1,12,-1378.160034,...,64754.585938,110542.594872,6662.970215,560835.375000,64754.585938,110542.594872,6662.970215,560835.375000,25,3
3,100004,1.333333,0.577350,1.0,2.0,2.000000,1.000000,1,3,-754.000000,...,7096.154785,3011.871719,5357.250000,10573.964844,7096.154785,3011.871719,5357.250000,10573.964844,3,1
4,100005,1.111111,0.333333,1.0,2.0,5.000000,2.738613,1,9,-586.000000,...,6240.205078,4281.014648,4813.200195,17656.244141,6240.205078,4281.014648,4813.200195,17656.244141,9,1


In [96]:
# infをnanへ置換
installments_ohe_agg = installments_ohe_agg.replace([np.inf, -np.inf], np.nan)

In [97]:
df_train = pd.merge(app_train, installments_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, previous_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, credit_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, pos_ohe_agg, on='SK_ID_CURR', how='left')
print(df_train.shape)
df_train.head()

(307506, 556)


,level_0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,19.0,1.0
1,1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,...,0.0,0.0,0.071429,0.0,0.000000,0.000000,0.0,0.0,28.0,3.0
2,2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,...,0.0,0.0,0.250000,0.0,0.000000,0.000000,0.0,0.0,4.0,1.0
3,3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,...,0.0,0.0,0.095238,0.0,0.047619,0.000000,0.0,0.0,21.0,3.0
4,4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,...,0.0,0.0,0.045455,0.0,0.000000,0.015152,0.0,0.0,66.0,5.0


In [98]:
# テーブル結合
df_test = pd.merge(app_test, installments_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, previous_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, credit_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, pos_ohe_agg, on='SK_ID_CURR', how='left')
print(df_test.shape)
df_test.head()

(48744, 555)


,level_0,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,307511,100001,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,...,0.0,0.0,0.222222,0.0,0.0,0.000000,0.0,0.0,9.0,2.0
1,307512,100005,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,...,0.0,0.0,0.090909,0.0,0.0,0.090909,0.0,0.0,11.0,1.0
2,307513,100013,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,...,0.0,0.0,0.083333,0.0,0.0,0.027778,0.0,0.0,36.0,3.0
3,307514,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,...,0.0,0.0,0.064516,0.0,0.0,0.000000,0.0,0.0,31.0,2.0
4,307515,100038,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,...,0.0,0.0,0.076923,0.0,0.0,0.000000,0.0,0.0,13.0,1.0


In [99]:
### 処理対象の列名を配列化
#encoding_columns = []
encoding_columns = list(df_test.select_dtypes(include='number').columns)

#for col in df_test.columns:
#    print(col)
#    encoding_columns.append(col)
    
encoding_columns.remove('SK_ID_CURR')

In [100]:
### Yao-Jhonson変換
for column in encoding_columns:
    modules.process_yj_convert(df_train, df_test, column, [column])

df_train.head()

,level_0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,-2.196836,100002,1.0,-0.653131,0.647085,-0.284046,0.026932,-0.324380,0.047085,1.597501,...,-0.117324,-0.006439,-1.882714,-0.03285,-0.122221,-0.493724,-0.002629,0.0,-0.187757,-1.255497
1,-2.196523,100003,0.0,-0.653131,1.213324,1.495396,0.744618,1.433694,-1.730864,-0.222757,...,-0.117324,-0.006439,-0.045923,-0.03285,-0.122221,-0.493724,-0.002629,0.0,0.304656,0.504269
2,-2.196256,100004,0.0,-0.653131,-1.654456,-1.648319,-2.082107,-1.533668,-0.863026,-0.716549,...,-0.117324,-0.006439,2.215187,-0.03285,-0.122221,-0.493724,-0.002629,0.0,-2.038472,-1.255497
3,-2.196014,100006,0.0,-0.653131,-0.176134,-0.635707,0.377928,-0.549543,-1.112439,-0.707904,...,-0.117324,-0.006439,0.405962,-0.03285,8.181937,-0.493724,-0.002629,0.0,-0.061302,0.504269
4,-2.195789,100007,0.0,-0.653131,-0.394983,0.042350,-0.198042,0.210360,0.799714,-0.901463,...,-0.117324,-0.006439,-0.620243,-0.03285,-0.122221,1.480653,-0.002629,0.0,1.408833,1.246159


In [101]:
### indexを設定
df_train = df_train.set_index('SK_ID_CURR')
df_test = df_test.set_index('SK_ID_CURR')
df_train.head()

,level_0,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,-2.196836,1.0,-0.653131,0.647085,-0.284046,0.026932,-0.324380,0.047085,1.597501,0.846146,...,-0.117324,-0.006439,-1.882714,-0.03285,-0.122221,-0.493724,-0.002629,0.0,-0.187757,-1.255497
100003,-2.196523,0.0,-0.653131,1.213324,1.495396,0.744618,1.433694,-1.730864,-0.222757,0.295507,...,-0.117324,-0.006439,-0.045923,-0.03285,-0.122221,-0.493724,-0.002629,0.0,0.304656,0.504269
100004,-2.196256,0.0,-0.653131,-1.654456,-1.648319,-2.082107,-1.533668,-0.863026,-0.716549,1.649642,...,-0.117324,-0.006439,2.215187,-0.03285,-0.122221,-0.493724,-0.002629,0.0,-2.038472,-1.255497
100006,-2.196014,0.0,-0.653131,-0.176134,-0.635707,0.377928,-0.549543,-1.112439,-0.707904,-0.647130,...,-0.117324,-0.006439,0.405962,-0.03285,8.181937,-0.493724,-0.002629,0.0,-0.061302,0.504269
100007,-2.195789,0.0,-0.653131,-0.394983,0.042350,-0.198042,0.210360,0.799714,-0.901463,-0.646418,...,-0.117324,-0.006439,-0.620243,-0.03285,-0.122221,1.480653,-0.002629,0.0,1.408833,1.246159


In [102]:
### 標準化
for column in encoding_columns:
    num_cols = []
    num_cols.append(column)
    result = modules.process_standardization(df_train, df_test, num_cols)
        
df_train.head()

,level_0,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,-2.196836,1.0,-0.653131,0.647085,-0.284046,0.026932,-0.324380,0.047085,1.597501,0.846146,...,-0.117324,-0.006439,-1.882714,-0.03285,-0.122221,-0.493724,-0.002629,0.0,-0.187757,-1.255497
100003,-2.196523,0.0,-0.653131,1.213324,1.495396,0.744618,1.433694,-1.730864,-0.222757,0.295507,...,-0.117324,-0.006439,-0.045923,-0.03285,-0.122221,-0.493724,-0.002629,0.0,0.304656,0.504269
100004,-2.196256,0.0,-0.653131,-1.654456,-1.648319,-2.082107,-1.533668,-0.863026,-0.716549,1.649642,...,-0.117324,-0.006439,2.215187,-0.03285,-0.122221,-0.493724,-0.002629,0.0,-2.038472,-1.255497
100006,-2.196014,0.0,-0.653131,-0.176134,-0.635707,0.377928,-0.549543,-1.112439,-0.707904,-0.647130,...,-0.117324,-0.006439,0.405962,-0.03285,8.181937,-0.493724,-0.002629,0.0,-0.061302,0.504269
100007,-2.195789,0.0,-0.653131,-0.394983,0.042350,-0.198042,0.210360,0.799714,-0.901463,-0.646418,...,-0.117324,-0.006439,-0.620243,-0.03285,-0.122221,1.480653,-0.002629,0.0,1.408833,1.246159


In [103]:
### indexを削除
df_train = df_train.reset_index()
df_test = df_test.reset_index()
df_train.head()

,SK_ID_CURR,level_0,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100002,-2.196836,1.0,-0.653131,0.647085,-0.284046,0.026932,-0.324380,0.047085,1.597501,...,-0.117324,-0.006439,-1.882714,-0.03285,-0.122221,-0.493724,-0.002629,0.0,-0.187757,-1.255497
1,100003,-2.196523,0.0,-0.653131,1.213324,1.495396,0.744618,1.433694,-1.730864,-0.222757,...,-0.117324,-0.006439,-0.045923,-0.03285,-0.122221,-0.493724,-0.002629,0.0,0.304656,0.504269
2,100004,-2.196256,0.0,-0.653131,-1.654456,-1.648319,-2.082107,-1.533668,-0.863026,-0.716549,...,-0.117324,-0.006439,2.215187,-0.03285,-0.122221,-0.493724,-0.002629,0.0,-2.038472,-1.255497
3,100006,-2.196014,0.0,-0.653131,-0.176134,-0.635707,0.377928,-0.549543,-1.112439,-0.707904,...,-0.117324,-0.006439,0.405962,-0.03285,8.181937,-0.493724,-0.002629,0.0,-0.061302,0.504269
4,100007,-2.195789,0.0,-0.653131,-0.394983,0.042350,-0.198042,0.210360,0.799714,-0.901463,...,-0.117324,-0.006439,-0.620243,-0.03285,-0.122221,1.480653,-0.002629,0.0,1.408833,1.246159


In [104]:
### メモリサイズを縮小
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()

Memory usage of dataframe is 857.49 MB
Memory usage of optimization is 683893472.00 MB
Decreased by -79754795.0%
Memory usage of dataframe is 135.74 MB
Memory usage of optimization is 108211808.00 MB
Decreased by -79720426.7%


,SK_ID_CURR,level_0,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100002,-2.197266,1.0,-0.65332,0.646973,-0.283936,0.026932,-0.324463,0.047089,1.597656,...,-0.11731,-0.006439,-1.882812,-0.032837,-0.122192,-0.493652,-0.002628,0.0,-0.187744,-1.255859
1,100003,-2.197266,0.0,-0.65332,1.212891,1.495117,0.744629,1.433594,-1.730469,-0.222778,...,-0.11731,-0.006439,-0.045929,-0.032837,-0.122192,-0.493652,-0.002628,0.0,0.304688,0.504395
2,100004,-2.195312,0.0,-0.65332,-1.654297,-1.648438,-2.082031,-1.533203,-0.862793,-0.716309,...,-0.11731,-0.006439,2.214844,-0.032837,-0.122192,-0.493652,-0.002628,0.0,-2.039062,-1.255859
3,100006,-2.195312,0.0,-0.65332,-0.176147,-0.635742,0.377930,-0.549316,-1.112305,-0.708008,...,-0.11731,-0.006439,0.406006,-0.032837,8.179688,-0.493652,-0.002628,0.0,-0.061310,0.504395
4,100007,-2.195312,0.0,-0.65332,-0.395020,0.042358,-0.197998,0.210327,0.799805,-0.901367,...,-0.11731,-0.006439,-0.620117,-0.032837,-0.122192,1.480469,-0.002628,0.0,1.409180,1.246094


In [105]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=="O":
        x_train[col] = x_train[col].astype('category')

In [106]:
baseline = Baseline('SK_ID_CURR')

train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246004, 554) (61502, 554)
[LightGBM] [Warning] num_threads is set with n_jobs=-1, nthread=-1 will be ignored. Current value: num_threads=-1
[100]	training's auc: 0.769562	valid_1's auc: 0.749374
[200]	training's auc: 0.783646	valid_1's auc: 0.75811
[300]	training's auc: 0.794845	valid_1's auc: 0.763944
[400]	training's auc: 0.804609	valid_1's auc: 0.769054
[500]	training's auc: 0.812719	valid_1's auc: 0.772428
[600]	training's auc: 0.819809	valid_1's auc: 0.77479
[700]	training's auc: 0.826355	valid_1's auc: 0.776579
[800]	training's auc: 0.832135	valid_1's auc: 0.777857
[900]	training's auc: 0.837694	valid_1's auc: 0.778799
[1000]	training's auc: 0.84296	valid_1's auc: 0.779523
[1100]	training's auc: 0.847916	valid_1's auc: 0.780065
[1200]	training's auc: 0.852556	valid_1's auc: 0.780465
[1300]	training's auc: 0.856993	valid_1's auc: 0.780728
[1400]	training's auc: 0.861177	valid_1's auc: 0.780911
[1500]	training's auc: 0.865191	valid_1's a

In [107]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
208,EXT_SOURCE_mean,330574.376484,4399.960123
358,NEW_EXT_SOURCES_GEO,145196.567763,6193.807550
360,NEW_EXT_SOURCES_MEDIAN,64948.744166,4642.014584
352,NEW_CREDIT_TO_ANNUITY_RATIO,49234.331459,1215.534980
114,ANNUITY_div_CREDIT,32988.476494,1388.063109
82,AMT_PAYMENT_min,30788.047127,1251.099828
205,EXT_SOURCE_3,26426.145246,3059.957168
543,app_EXT_SOURCE_2_times_EXT_SOURCE_3,26146.694105,2519.035780
157,CNT_INSTALMENT_FUTURE_mean,24800.313419,708.414454
207,EXT_SOURCE_max,24018.998301,2337.471801


In [108]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [109]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [110]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_featureEngineering5.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.032827
1,100005,0.080884
2,100013,0.031239
3,100028,0.042919
4,100038,0.149888


### bureauの読み込み

In [111]:
bureau = bureau_origin.copy()
bureau = baseline.reduce_mem_usage(bureau)
print(bureau.shape)
bureau.head()

Memory usage of dataframe is 222.62 MB
Memory usage of optimization is 125299372.00 MB
Decreased by -56283785.5%
(1716428, 17)


,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [112]:
### 
for col in bureau.columns:
    bureau[col] = bureau[col].replace('\s', '_', regex=True)
    bureau[col] = bureau[col].replace(':', '_', regex=True)
    bureau[col] = bureau[col].replace('/', '_', regex=True)
    
bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency_1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer_credit,-131,NaN
1,215354,5714463,Active,currency_1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit_card,-20,NaN
2,215354,5714464,Active,currency_1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer_credit,-16,NaN
3,215354,5714465,Active,currency_1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit_card,-16,NaN
4,215354,5714466,Active,currency_1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer_credit,-21,NaN


In [113]:
# one-hotencoding(処理対象カラムを取得)
encoding_columns = []

for col in bureau.columns:
    if bureau[col].dtype=="O":
        print(col)
        encoding_columns.append(col)

CREDIT_ACTIVE
CREDIT_CURRENCY
CREDIT_TYPE


In [114]:
# one-hot-encoding
bureau_ohe = pd.get_dummies(bureau, columns=['CREDIT_ACTIVE'], dummy_na=True)
col_ohe = sorted(list(set(bureau_ohe.columns) - set(bureau.columns)))

bureau_ohe = pd.get_dummies(bureau_ohe, columns=['CREDIT_CURRENCY'], dummy_na=True)
col_ohe = sorted(list(set(bureau_ohe.columns) - set(bureau.columns)))

bureau_ohe = pd.get_dummies(bureau_ohe, columns=['CREDIT_TYPE'], dummy_na=True)
col_ohe = sorted(list(set(bureau_ohe.columns) - set(bureau.columns)))

print(len(col_ohe))
col_ohe

26


['CREDIT_ACTIVE_Active',
 'CREDIT_ACTIVE_Bad_debt',
 'CREDIT_ACTIVE_Closed',
 'CREDIT_ACTIVE_Sold',
 'CREDIT_ACTIVE_nan',
 'CREDIT_CURRENCY_currency_1',
 'CREDIT_CURRENCY_currency_2',
 'CREDIT_CURRENCY_currency_3',
 'CREDIT_CURRENCY_currency_4',
 'CREDIT_CURRENCY_nan',
 'CREDIT_TYPE_Another_type_of_loan',
 'CREDIT_TYPE_Car_loan',
 'CREDIT_TYPE_Cash_loan_(non-earmarked)',
 'CREDIT_TYPE_Consumer_credit',
 'CREDIT_TYPE_Credit_card',
 'CREDIT_TYPE_Interbank_credit',
 'CREDIT_TYPE_Loan_for_business_development',
 'CREDIT_TYPE_Loan_for_purchase_of_shares_(margin_lending)',
 'CREDIT_TYPE_Loan_for_the_purchase_of_equipment',
 'CREDIT_TYPE_Loan_for_working_capital_replenishment',
 'CREDIT_TYPE_Microloan',
 'CREDIT_TYPE_Mobile_operator_loan',
 'CREDIT_TYPE_Mortgage',
 'CREDIT_TYPE_Real_estate_loan',
 'CREDIT_TYPE_Unknown_type_of_loan',
 'CREDIT_TYPE_nan']

In [115]:
bureau_ohe = bureau_ohe.rename(columns={'SK_ID_BUREAU': 'SK_ID_BUREAU_BUREAU'})

In [116]:
bureau_ohe.head()

,SK_ID_CURR,SK_ID_BUREAU_BUREAU,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,...,CREDIT_TYPE_Loan_for_business_development,CREDIT_TYPE_Loan_for_purchase_of_shares_(margin_lending),CREDIT_TYPE_Loan_for_the_purchase_of_equipment,CREDIT_TYPE_Loan_for_working_capital_replenishment,CREDIT_TYPE_Microloan,CREDIT_TYPE_Mobile_operator_loan,CREDIT_TYPE_Mortgage,CREDIT_TYPE_Real_estate_loan,CREDIT_TYPE_Unknown_type_of_loan,CREDIT_TYPE_nan
0,215354,5714462,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,215354,5714463,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,...,0,0,0,0,0,0,0,0,0,0
2,215354,5714464,-203,0,528.0,NaN,NaN,0,464323.5,NaN,...,0,0,0,0,0,0,0,0,0,0
3,215354,5714465,-203,0,NaN,NaN,NaN,0,90000.0,NaN,...,0,0,0,0,0,0,0,0,0,0
4,215354,5714466,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,...,0,0,0,0,0,0,0,0,0,0


In [117]:
bureau_ohe_agg = bureau_ohe.groupby('SK_ID_CURR').agg(
    {
        # 数値変数の集約
        'DAYS_CREDIT': ['mean', 'std', 'min', 'max', 'var']
        ,'CREDIT_DAY_OVERDUE': ['mean', 'std', 'min', 'max']
        ,'DAYS_CREDIT_ENDDATE': ['mean', 'std', 'min', 'max']
        ,'DAYS_ENDDATE_FACT': ['mean', 'std', 'min', 'max']
        ,'AMT_CREDIT_MAX_OVERDUE': ['mean', 'std', 'min', 'max']
        ,'CNT_CREDIT_PROLONG': ['mean', 'std', 'min', 'max']
        ,'AMT_CREDIT_SUM': ['mean', 'std', 'min', 'max', 'sum']
        ,'AMT_CREDIT_SUM_DEBT': ['mean', 'std', 'min', 'max', 'sum']
        ,'AMT_CREDIT_SUM_LIMIT': ['mean', 'std', 'min', 'max']
        ,'AMT_CREDIT_SUM_OVERDUE': ['mean', 'std', 'min', 'max']
        ,'DAYS_CREDIT_UPDATE': ['mean', 'std', 'min', 'max']
        ,'AMT_ANNUITY': ['mean', 'std', 'min', 'max']
        # 
        ,'CREDIT_TYPE_Another_type_of_loan': ['mean']
        ,'CREDIT_TYPE_Car_loan': ['mean']
        ,'CREDIT_TYPE_Cash_loan_(non-earmarked)': ['mean']
        ,'CREDIT_TYPE_Consumer_credit': ['mean']
        ,'CREDIT_TYPE_Credit_card': ['mean']
        ,'CREDIT_TYPE_Interbank_credit': ['mean']
        ,'CREDIT_TYPE_Loan_for_business_development': ['mean']
        ,'CREDIT_TYPE_Loan_for_purchase_of_shares_(margin_lending)': ['mean']
        ,'CREDIT_TYPE_Loan_for_the_purchase_of_equipment': ['mean']
        ,'CREDIT_TYPE_Loan_for_working_capital_replenishment': ['mean']
        ,'CREDIT_TYPE_Microloan': ['mean']
        ,'CREDIT_TYPE_Mobile_operator_loan': ['mean']
        ,'CREDIT_TYPE_Mortgage': ['mean']
        ,'CREDIT_TYPE_Real_estate_loan': ['mean']
        ,'CREDIT_TYPE_Unknown_type_of_loan': ['mean']
        ,'CREDIT_TYPE_nan': ['mean']
        ,'CREDIT_ACTIVE_Active': ['mean']
        ,'CREDIT_ACTIVE_Bad_debt': ['mean']
        ,'CREDIT_ACTIVE_Closed': ['mean']
        ,'CREDIT_ACTIVE_Sold': ['mean']
        ,'CREDIT_ACTIVE_nan': ['mean']
        ,'CREDIT_CURRENCY_currency_1': ['mean']
        ,'CREDIT_CURRENCY_currency_2': ['mean']
        ,'CREDIT_CURRENCY_currency_3': ['mean']
        ,'CREDIT_CURRENCY_currency_4': ['mean']
        ,'CREDIT_CURRENCY_nan': ['mean']
        ,'CREDIT_TYPE_Another_type_of_loan': ['mean']
        ,'CREDIT_TYPE_Car_loan': ['mean']
        ,'CREDIT_TYPE_Cash_loan_(non-earmarked)': ['mean']
        ,'CREDIT_TYPE_Consumer_credit': ['mean']
        ,'CREDIT_TYPE_Credit_card': ['mean']
        ,'CREDIT_TYPE_Interbank_credit': ['mean']
        ,'CREDIT_TYPE_Loan_for_business_development': ['mean']
        ,'CREDIT_TYPE_Loan_for_purchase_of_shares_(margin_lending)': ['mean']
        ,'CREDIT_TYPE_Loan_for_the_purchase_of_equipment': ['mean']
        ,'CREDIT_TYPE_Loan_for_working_capital_replenishment': ['mean']
        ,'CREDIT_TYPE_Microloan': ['mean']
        ,'CREDIT_TYPE_Mobile_operator_loan': ['mean']
        ,'CREDIT_TYPE_Mortgage': ['mean']
        ,'CREDIT_TYPE_Real_estate_loan': ['mean']
        ,'CREDIT_TYPE_Unknown_type_of_loan': ['mean']
        ,'CREDIT_TYPE_nan': ['mean']
        # IDのユニーク数をカウント
        ,'SK_ID_BUREAU_BUREAU': ["count", "nunique"]
    }
)

# カラム名の付与
bureau_ohe_agg.columns = [i + "_" + j for i,j in bureau_ohe_agg.columns]
bureau_ohe_agg = bureau_ohe_agg.reset_index(drop=False)

#
bureau_ohe_agg['debt_credit_ratio_None'] = bureau_ohe_agg['AMT_CREDIT_SUM_DEBT_sum'] / bureau_ohe_agg['AMT_CREDIT_SUM_sum']

print(bureau_ohe_agg.shape)
bureau_ohe_agg.head()

(305811, 81)


,SK_ID_CURR,DAYS_CREDIT_mean,DAYS_CREDIT_std,DAYS_CREDIT_min,DAYS_CREDIT_max,DAYS_CREDIT_var,CREDIT_DAY_OVERDUE_mean,CREDIT_DAY_OVERDUE_std,CREDIT_DAY_OVERDUE_min,CREDIT_DAY_OVERDUE_max,...,CREDIT_ACTIVE_Sold_mean,CREDIT_ACTIVE_nan_mean,CREDIT_CURRENCY_currency_1_mean,CREDIT_CURRENCY_currency_2_mean,CREDIT_CURRENCY_currency_3_mean,CREDIT_CURRENCY_currency_4_mean,CREDIT_CURRENCY_nan_mean,SK_ID_BUREAU_BUREAU_count,SK_ID_BUREAU_BUREAU_nunique,debt_credit_ratio_None
0,100001,-735.000000,489.942514,-1572,-49,240043.666667,0.0,0.0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,7,7,0.410555
1,100002,-874.000000,431.451040,-1437,-103,186150.000000,0.0,0.0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,8,8,0.284122
2,100003,-1400.750000,909.826128,-2586,-606,827783.583333,0.0,0.0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4,4,0.000000
3,100004,-867.000000,649.124025,-1326,-408,421362.000000,0.0,0.0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2,2,0.000000
4,100005,-190.666667,162.297053,-373,-62,26340.333333,0.0,0.0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3,3,0.864992


In [118]:
# infをnanへ置換
bureau_ohe_agg = bureau_ohe_agg.replace([np.inf, -np.inf], np.nan)

In [119]:
df_train = pd.merge(app_train, bureau_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, installments_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, previous_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, credit_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, pos_ohe_agg, on='SK_ID_CURR', how='left')
print(df_train.shape)
df_train.head()

(307506, 636)


,level_0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,19.0,1.0
1,1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,...,0.0,0.0,0.071429,0.0,0.000000,0.000000,0.0,0.0,28.0,3.0
2,2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,...,0.0,0.0,0.250000,0.0,0.000000,0.000000,0.0,0.0,4.0,1.0
3,3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,...,0.0,0.0,0.095238,0.0,0.047619,0.000000,0.0,0.0,21.0,3.0
4,4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,...,0.0,0.0,0.045455,0.0,0.000000,0.015152,0.0,0.0,66.0,5.0


In [120]:
df_test = pd.merge(app_test, bureau_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, installments_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, previous_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, credit_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, pos_ohe_agg, on='SK_ID_CURR', how='left')
print(df_test.shape)
df_test.head()

(48744, 635)


,level_0,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,307511,100001,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,...,0.0,0.0,0.222222,0.0,0.0,0.000000,0.0,0.0,9.0,2.0
1,307512,100005,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,...,0.0,0.0,0.090909,0.0,0.0,0.090909,0.0,0.0,11.0,1.0
2,307513,100013,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,...,0.0,0.0,0.083333,0.0,0.0,0.027778,0.0,0.0,36.0,3.0
3,307514,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,...,0.0,0.0,0.064516,0.0,0.0,0.000000,0.0,0.0,31.0,2.0
4,307515,100038,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,...,0.0,0.0,0.076923,0.0,0.0,0.000000,0.0,0.0,13.0,1.0


In [121]:
### 処理対象の列名を配列化
#encoding_columns = []
encoding_columns = list(df_test.select_dtypes(include='number').columns)

#for col in df_test.columns:
#    print(col)
#    encoding_columns.append(col)
    
encoding_columns.remove('SK_ID_CURR')
encoding_columns.remove('EXT_SOURCE_3')
encoding_columns.remove('EXT_SOURCE_2')
encoding_columns.remove('EXT_SOURCE_1')

In [122]:
### Yao-Jhonson変換
for column in encoding_columns:
    modules.process_yj_convert(df_train, df_test, column, [column])

df_train.head()

,level_0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,-2.196836,100002,1.0,-0.653131,0.647085,-0.284046,0.026932,-0.324380,0.047085,1.597501,...,-0.117324,-0.006439,-1.882714,-0.03285,-0.122221,-0.493724,-0.002629,0.0,-0.187757,-1.255497
1,-2.196523,100003,0.0,-0.653131,1.213324,1.495396,0.744618,1.433694,-1.730864,-0.222757,...,-0.117324,-0.006439,-0.045923,-0.03285,-0.122221,-0.493724,-0.002629,0.0,0.304656,0.504269
2,-2.196256,100004,0.0,-0.653131,-1.654456,-1.648319,-2.082107,-1.533668,-0.863026,-0.716549,...,-0.117324,-0.006439,2.215187,-0.03285,-0.122221,-0.493724,-0.002629,0.0,-2.038472,-1.255497
3,-2.196014,100006,0.0,-0.653131,-0.176134,-0.635707,0.377928,-0.549543,-1.112439,-0.707904,...,-0.117324,-0.006439,0.405962,-0.03285,8.181937,-0.493724,-0.002629,0.0,-0.061302,0.504269
4,-2.195789,100007,0.0,-0.653131,-0.394983,0.042350,-0.198042,0.210360,0.799714,-0.901463,...,-0.117324,-0.006439,-0.620243,-0.03285,-0.122221,1.480653,-0.002629,0.0,1.408833,1.246159


In [123]:
encoding_columns.append('EXT_SOURCE_3')
encoding_columns.append('EXT_SOURCE_2')
encoding_columns.append('EXT_SOURCE_1')

In [124]:
### indexを設定
df_train = df_train.set_index('SK_ID_CURR')
df_test = df_test.set_index('SK_ID_CURR')
df_train.head()

,level_0,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,-2.196836,1.0,-0.653131,0.647085,-0.284046,0.026932,-0.324380,0.047085,1.597501,0.846146,...,-0.117324,-0.006439,-1.882714,-0.03285,-0.122221,-0.493724,-0.002629,0.0,-0.187757,-1.255497
100003,-2.196523,0.0,-0.653131,1.213324,1.495396,0.744618,1.433694,-1.730864,-0.222757,0.295507,...,-0.117324,-0.006439,-0.045923,-0.03285,-0.122221,-0.493724,-0.002629,0.0,0.304656,0.504269
100004,-2.196256,0.0,-0.653131,-1.654456,-1.648319,-2.082107,-1.533668,-0.863026,-0.716549,1.649642,...,-0.117324,-0.006439,2.215187,-0.03285,-0.122221,-0.493724,-0.002629,0.0,-2.038472,-1.255497
100006,-2.196014,0.0,-0.653131,-0.176134,-0.635707,0.377928,-0.549543,-1.112439,-0.707904,-0.647130,...,-0.117324,-0.006439,0.405962,-0.03285,8.181937,-0.493724,-0.002629,0.0,-0.061302,0.504269
100007,-2.195789,0.0,-0.653131,-0.394983,0.042350,-0.198042,0.210360,0.799714,-0.901463,-0.646418,...,-0.117324,-0.006439,-0.620243,-0.03285,-0.122221,1.480653,-0.002629,0.0,1.408833,1.246159


In [125]:
### 標準化
for column in encoding_columns:
    num_cols = []
    num_cols.append(column)
    result = modules.process_standardization(df_train, df_test, num_cols)
        
df_train.head()

,level_0,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,-2.196836,1.0,-0.653131,0.647085,-0.284046,0.026932,-0.324380,0.047085,1.597501,0.846146,...,-0.117324,-0.006439,-1.882714,-0.03285,-0.122221,-0.493724,-0.002629,0.0,-0.187757,-1.255497
100003,-2.196523,0.0,-0.653131,1.213324,1.495396,0.744618,1.433694,-1.730864,-0.222757,0.295507,...,-0.117324,-0.006439,-0.045923,-0.03285,-0.122221,-0.493724,-0.002629,0.0,0.304656,0.504269
100004,-2.196256,0.0,-0.653131,-1.654456,-1.648319,-2.082107,-1.533668,-0.863026,-0.716549,1.649642,...,-0.117324,-0.006439,2.215187,-0.03285,-0.122221,-0.493724,-0.002629,0.0,-2.038472,-1.255497
100006,-2.196014,0.0,-0.653131,-0.176134,-0.635707,0.377928,-0.549543,-1.112439,-0.707904,-0.647130,...,-0.117324,-0.006439,0.405962,-0.03285,8.181937,-0.493724,-0.002629,0.0,-0.061302,0.504269
100007,-2.195789,0.0,-0.653131,-0.394983,0.042350,-0.198042,0.210360,0.799714,-0.901463,-0.646418,...,-0.117324,-0.006439,-0.620243,-0.03285,-0.122221,1.480653,-0.002629,0.0,1.408833,1.246159


In [126]:
### indexを削除
df_train = df_train.reset_index()
df_test = df_test.reset_index()
df_train.head()

,SK_ID_CURR,level_0,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100002,-2.196836,1.0,-0.653131,0.647085,-0.284046,0.026932,-0.324380,0.047085,1.597501,...,-0.117324,-0.006439,-1.882714,-0.03285,-0.122221,-0.493724,-0.002629,0.0,-0.187757,-1.255497
1,100003,-2.196523,0.0,-0.653131,1.213324,1.495396,0.744618,1.433694,-1.730864,-0.222757,...,-0.117324,-0.006439,-0.045923,-0.03285,-0.122221,-0.493724,-0.002629,0.0,0.304656,0.504269
2,100004,-2.196256,0.0,-0.653131,-1.654456,-1.648319,-2.082107,-1.533668,-0.863026,-0.716549,...,-0.117324,-0.006439,2.215187,-0.03285,-0.122221,-0.493724,-0.002629,0.0,-2.038472,-1.255497
3,100006,-2.196014,0.0,-0.653131,-0.176134,-0.635707,0.377928,-0.549543,-1.112439,-0.707904,...,-0.117324,-0.006439,0.405962,-0.03285,8.181937,-0.493724,-0.002629,0.0,-0.061302,0.504269
4,100007,-2.195789,0.0,-0.653131,-0.394983,0.042350,-0.198042,0.210360,0.799714,-0.901463,...,-0.117324,-0.006439,-0.620243,-0.03285,-0.122221,1.480653,-0.002629,0.0,1.408833,1.246159


In [127]:
### メモリサイズを縮小
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()


Memory usage of dataframe is 1045.18 MB
Memory usage of optimization is 782295392.00 MB
Decreased by -74847752.7%
Memory usage of dataframe is 165.49 MB
Memory usage of optimization is 123809888.00 MB
Decreased by -74814052.5%


,SK_ID_CURR,level_0,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100002,-2.197266,1.0,-0.65332,0.646973,-0.283936,0.026932,-0.324463,0.047089,1.597656,...,-0.11731,-0.006439,-1.882812,-0.032837,-0.122192,-0.493652,-0.002628,0.0,-0.187744,-1.255859
1,100003,-2.197266,0.0,-0.65332,1.212891,1.495117,0.744629,1.433594,-1.730469,-0.222778,...,-0.11731,-0.006439,-0.045929,-0.032837,-0.122192,-0.493652,-0.002628,0.0,0.304688,0.504395
2,100004,-2.195312,0.0,-0.65332,-1.654297,-1.648438,-2.082031,-1.533203,-0.862793,-0.716309,...,-0.11731,-0.006439,2.214844,-0.032837,-0.122192,-0.493652,-0.002628,0.0,-2.039062,-1.255859
3,100006,-2.195312,0.0,-0.65332,-0.176147,-0.635742,0.377930,-0.549316,-1.112305,-0.708008,...,-0.11731,-0.006439,0.406006,-0.032837,8.179688,-0.493652,-0.002628,0.0,-0.061310,0.504395
4,100007,-2.195312,0.0,-0.65332,-0.395020,0.042358,-0.197998,0.210327,0.799805,-0.901367,...,-0.11731,-0.006439,-0.620117,-0.032837,-0.122192,1.480469,-0.002628,0.0,1.409180,1.246094


In [128]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=="O":
        x_train[col] = x_train[col].astype('category')

In [129]:
baseline = Baseline('SK_ID_CURR')

train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246004, 634) (61502, 634)
[LightGBM] [Warning] num_threads is set with n_jobs=-1, nthread=-1 will be ignored. Current value: num_threads=-1
[100]	training's auc: 0.771469	valid_1's auc: 0.750464
[200]	training's auc: 0.786264	valid_1's auc: 0.759437
[300]	training's auc: 0.798639	valid_1's auc: 0.766244
[400]	training's auc: 0.809305	valid_1's auc: 0.771954
[500]	training's auc: 0.817924	valid_1's auc: 0.775594
[600]	training's auc: 0.825693	valid_1's auc: 0.778371
[700]	training's auc: 0.832527	valid_1's auc: 0.780394
[800]	training's auc: 0.838889	valid_1's auc: 0.781955
[900]	training's auc: 0.84473	valid_1's auc: 0.78318
[1000]	training's auc: 0.850215	valid_1's auc: 0.784067
[1100]	training's auc: 0.855497	valid_1's auc: 0.784846
[1200]	training's auc: 0.860479	valid_1's auc: 0.785357
[1300]	training's auc: 0.865289	valid_1's auc: 0.786026
[1400]	training's auc: 0.869832	valid_1's auc: 0.786438
[1500]	training's auc: 0.874061	valid_1's 

In [130]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
285,EXT_SOURCE_mean,311643.472622,12531.853492
435,NEW_EXT_SOURCES_GEO,171759.982530,8809.139185
437,NEW_EXT_SOURCES_MEDIAN,62687.475443,20943.737260
429,NEW_CREDIT_TO_ANNUITY_RATIO,44553.827681,1214.128747
140,ANNUITY_div_CREDIT,32198.376199,2721.713249
108,AMT_PAYMENT_min,30684.801762,1422.261737
284,EXT_SOURCE_max,28079.097182,4736.985600
632,debt_credit_ratio_None,24308.265761,1229.056281
187,CNT_INSTALMENT_FUTURE_mean,24098.291150,1221.278018
622,app_EXT_SOURCE_2_times_EXT_SOURCE_3,21271.383354,3227.067310


In [131]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [132]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [133]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_featureEngineering6.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.034923
1,100005,0.109607
2,100013,0.036768
3,100028,0.034721
4,100038,0.153525


### mean mean 

In [134]:
# consumer_annuity_to_max_installment_ratio
app_train_mean1 = pd.read_csv('./home-credit-default-risk/datasets/train_hc_only_consumer_annuity_to_max_installment_ratio.csv')
app_test_mean1 =  pd.read_csv('./home-credit-default-risk/datasets/test_hc_only_consumer_annuity_to_max_installment_ratio.csv')
app_train_mean1 = app_train_mean1.rename(columns={'sk_id_curr': 'SK_ID_CURR'})
app_test_mean1 = app_test_mean1.rename(columns={'sk_id_curr': 'SK_ID_CURR'})

### installment_payment_ratio_1000_mean_mean
#### consumer_installment_payment_ratio_1000_mean_mean
app_train_mean2 = pd.read_csv('./home-credit-default-risk/datasets/train_hc_only_consumer_installment_payment_ratio_1000_mean_mean.csv')
app_test_mean2 =  pd.read_csv('./home-credit-default-risk/datasets/test_hc_only_consumer_installment_payment_ratio_1000_mean_mean.csv')
app_train_mean2 = app_train_mean2.rename(columns={'sk_id_curr': 'SK_ID_CURR'})
app_test_mean2 = app_test_mean2.rename(columns={'sk_id_curr': 'SK_ID_CURR'})


#### revo_installment_payment_ratio_1000_mean_mean
app_train_mean3 = pd.read_csv('./home-credit-default-risk/datasets/train_hc_only_revo_installment_payment_ratio_1000_mean_mean.csv')
app_test_mean3 =  pd.read_csv('./home-credit-default-risk/datasets/test_hc_only_revo_installment_payment_ratio_1000_mean_mean.csv')
app_train_mean3 = app_train_mean3.rename(columns={'sk_id_curr': 'SK_ID_CURR'})
app_test_mean3 = app_test_mean3.rename(columns={'sk_id_curr': 'SK_ID_CURR'})

#### installment_payment_ratio_1000_mean_mean
app_train_mean4 = pd.read_csv('./home-credit-default-risk/datasets/train_hc_only_cash_installment_payment_ratio_1000_mean_mean.csv')
app_test_mean4 =  pd.read_csv('./home-credit-default-risk/datasets/test_hc_only_cash_installment_payment_ratio_1000_mean_mean.csv')
app_train_mean4 = app_train_mean4.rename(columns={'sk_id_curr': 'SK_ID_CURR'})
app_test_mean4 = app_test_mean4.rename(columns={'sk_id_curr': 'SK_ID_CURR'})


In [135]:
# infをnanへ置換
app_train_mean1 = app_train_mean1.replace([np.inf, -np.inf], np.nan)
app_test_mean1 = app_train_mean1.replace([np.inf, -np.inf], np.nan)

app_train_mean2 = app_train_mean2.replace([np.inf, -np.inf], np.nan)
app_test_mean2 = app_test_mean2.replace([np.inf, -np.inf], np.nan)

app_train_mean3 = app_train_mean3.replace([np.inf, -np.inf], np.nan)
app_test_mean3 = app_test_mean3.replace([np.inf, -np.inf], np.nan)

app_train_mean4 = app_train_mean4.replace([np.inf, -np.inf], np.nan)
app_test_mean4 = app_train_mean4.replace([np.inf, -np.inf], np.nan)

In [136]:
#
#app_train_mean1 = app_train_mean1.fillna(0)
#app_test_mean1 = app_test_mean1.fillna(0)

#app_train_mean2 = app_train_mean2.fillna(0)
#app_test_mean2 = app_test_mean2.fillna(0)

#app_train_mean3 = app_train_mean3.fillna(0)
#app_test_mean3 = app_test_mean3.fillna(0)

#app_train_mean4 = app_train_mean4.fillna(0)
#app_test_mean4 = app_test_mean4.fillna(0)

In [137]:
df_train = pd.merge(app_train, bureau_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, installments_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, previous_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, credit_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, pos_ohe_agg, on='SK_ID_CURR', how='left')

df_train = pd.merge(df_train, app_train_mean1, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, app_train_mean2, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, app_train_mean3, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, app_train_mean4, on='SK_ID_CURR', how='left')

df_train = df_train.fillna(0)

print(df_train.shape)
df_train.head()

(307506, 640)


,level_0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique,consumer_annuity_to_max_installment_ratio,consumer_installment_payment_ratio_1000_mean_mean,revo_installment_payment_ratio_1000_mean_mean,installment_payment_ratio_1000_mean_mean
0,0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,...,0.000000,0.000000,0.0,0.0,19.0,1.0,-988.0000,0.0,0.0,0.0
1,1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,...,0.000000,0.000000,0.0,0.0,28.0,3.0,0.0000,0.0,0.0,0.0
2,2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,...,0.000000,0.000000,0.0,0.0,4.0,1.0,0.0000,0.0,0.0,0.0
3,3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,...,0.047619,0.000000,0.0,0.0,21.0,3.0,-2698.7273,0.0,0.0,0.0
4,4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,...,0.000000,0.015152,0.0,0.0,66.0,5.0,0.0000,0.0,0.0,0.0


In [138]:
df_test = pd.merge(app_test, bureau_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, installments_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, previous_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, credit_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, pos_ohe_agg, on='SK_ID_CURR', how='left')

df_test = pd.merge(df_test, app_test_mean1, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, app_test_mean2, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, app_test_mean3, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, app_test_mean4, on='SK_ID_CURR', how='left')

df_test = df_test.fillna(0)

print(df_test.shape)
df_test.head()

(48744, 639)


,level_0,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique,consumer_annuity_to_max_installment_ratio,consumer_installment_payment_ratio_1000_mean_mean,revo_installment_payment_ratio_1000_mean_mean,installment_payment_ratio_1000_mean_mean
0,307511,100001,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,...,0.0,0.000000,0.0,0.0,9.0,2.0,0.0,0.00000,0.000000,0.0
1,307512,100005,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,...,0.0,0.090909,0.0,0.0,11.0,1.0,0.0,0.00000,0.000000,0.0
2,307513,100013,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,...,0.0,0.027778,0.0,0.0,36.0,3.0,0.0,-1185.60375,0.000000,0.0
3,307514,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,...,0.0,0.000000,0.0,0.0,31.0,2.0,0.0,0.00000,-1148.027459,0.0
4,307515,100038,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,...,0.0,0.000000,0.0,0.0,13.0,1.0,0.0,0.00000,0.000000,0.0


In [139]:
# 処理対象の列名をリスト化(Rev/Cashのcolumnを取得する)
replace_columns = list(df_test.select_dtypes(include='number').columns)
replace_columns.remove('SK_ID_CURR')
replace_columns.remove('EXT_SOURCE_3')
replace_columns.remove('EXT_SOURCE_2')
replace_columns.remove('EXT_SOURCE_1')

In [140]:
### 欠損値を補完する()

#for column in processing_columns:
#    #print("Missing Value Completion Process: " + column + "\n")
#    result = modules.process_imputer(df_train, df_test, replace_columns, column)
#    if result != True:
#        print("Can not Process: " + column + "\n")

In [141]:
### infをnanへ置換
df_train = df_train.replace([np.inf, -np.inf], np.nan)
df_train.head()

,level_0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique,consumer_annuity_to_max_installment_ratio,consumer_installment_payment_ratio_1000_mean_mean,revo_installment_payment_ratio_1000_mean_mean,installment_payment_ratio_1000_mean_mean
0,0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,...,0.000000,0.000000,0.0,0.0,19.0,1.0,-988.0000,0.0,0.0,0.0
1,1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,...,0.000000,0.000000,0.0,0.0,28.0,3.0,0.0000,0.0,0.0,0.0
2,2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,...,0.000000,0.000000,0.0,0.0,4.0,1.0,0.0000,0.0,0.0,0.0
3,3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,...,0.047619,0.000000,0.0,0.0,21.0,3.0,-2698.7273,0.0,0.0,0.0
4,4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,...,0.000000,0.015152,0.0,0.0,66.0,5.0,0.0000,0.0,0.0,0.0


In [142]:
### 欠損値を補完する
#df_train = df_train.fillna(df_train.max() + (df_train.max() * 0.4))
#df_train.head()

In [143]:
### infをnanへ置換
df_test = df_test.replace([np.inf, -np.inf], np.nan)
df_test.head()

,level_0,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique,consumer_annuity_to_max_installment_ratio,consumer_installment_payment_ratio_1000_mean_mean,revo_installment_payment_ratio_1000_mean_mean,installment_payment_ratio_1000_mean_mean
0,307511,100001,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,...,0.0,0.000000,0.0,0.0,9.0,2.0,0.0,0.00000,0.000000,0.0
1,307512,100005,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,...,0.0,0.090909,0.0,0.0,11.0,1.0,0.0,0.00000,0.000000,0.0
2,307513,100013,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,...,0.0,0.027778,0.0,0.0,36.0,3.0,0.0,-1185.60375,0.000000,0.0
3,307514,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,...,0.0,0.000000,0.0,0.0,31.0,2.0,0.0,0.00000,-1148.027459,0.0
4,307515,100038,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,...,0.0,0.000000,0.0,0.0,13.0,1.0,0.0,0.00000,0.000000,0.0


In [144]:
### 欠損値補完
#df_test = df_test.fillna(df_test.max() + (df_test.max() * 0.4))
#df_test.head()

In [145]:
### 処理対象の列名を配列化
#encoding_columns = []
encoding_columns = list(df_test.select_dtypes(include='number').columns)

#for col in df_test.columns:
#    print(col)
#    encoding_columns.append(col)
    
encoding_columns.remove('SK_ID_CURR')
#encoding_columns.remove('EXT_SOURCE_3')
#encoding_columns.remove('EXT_SOURCE_2')
#encoding_columns.remove('EXT_SOURCE_1')

In [146]:
### Yao-Jhonson変換
for column in encoding_columns:
    modules.process_yj_convert(df_train, df_test, column, [column])

df_train.head()

Unexpected err=KeyboardInterrupt(), type(err)=<class 'KeyboardInterrupt'>


KeyboardInterrupt: 

In [ ]:
#encoding_columns.append('EXT_SOURCE_3')
#encoding_columns.append('EXT_SOURCE_2')
#encoding_columns.append('EXT_SOURCE_1')

In [ ]:
### indexを設定
df_train = df_train.set_index('SK_ID_CURR')
df_test = df_test.set_index('SK_ID_CURR')
df_train.head()

In [ ]:
### 標準化
for column in encoding_columns:
    num_cols = []
    num_cols.append(column)
    result = modules.process_standardization(df_train, df_test, num_cols)
        
df_train.head()

In [ ]:
### indexを削除
df_train = df_train.reset_index()
df_test = df_test.reset_index()
df_train.head()

In [ ]:
### メモリサイズを縮小
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()

In [ ]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=="O":
        x_train[col] = x_train[col].astype('category')

In [ ]:
baseline = Baseline('SK_ID_CURR')

train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

In [ ]:
imp.sort_values("imp", ascending=False)[:20]

In [ ]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [ ]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

In [ ]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_featureEngineering7.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

### 変数選択001

In [ ]:
length = int(len(df_train.columns) * 0.95)
length

In [ ]:
cols = list(imp.sort_values("imp", ascending=False)['col'][:length])
if 'TARGET' not in cols:
    print('aa')
    
if 'SK_ID_CURR' not in cols:
    print('bb')
    
train_columns = cols.copy()
train_columns.append('SK_ID_CURR')
train_columns.append('TARGET')

test_columns = cols.copy()
test_columns.append('SK_ID_CURR')

In [ ]:
df_train = df_train[train_columns]
df_test = df_test[test_columns]
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()

In [ ]:
### メモリサイズを縮小
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()

In [ ]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=="O":
        x_train[col] = x_train[col].astype('category')

In [ ]:
baseline = Baseline('SK_ID_CURR')

train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

In [ ]:
imp.sort_values("imp", ascending=False)[:20]

In [ ]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [ ]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

In [ ]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_featureEngineering8.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

## パラメータ・チューニング

In [ ]:
baseline = Baseline('SK_ID_CURR')

sampler = optuna.samplers.TPESampler(seed=123)
study = optuna.create_study(sampler=sampler, direction='maximize')
study.optimize(objective, n_trials=50, n_jobs=5)

In [ ]:
trial = study.best_trial
print("auc(best)={:.4f}".format(trial.value))
display(trial.params)

In [ ]:
params_best = trial.params
params_best.update(params_base)
display(params_best)

In [ ]:
train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5
                                    ,params=params_best)

In [ ]:
# 推論用データセットの作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

# predict
test_pred = baseline.predict_lgb(x_test
                       ,id_test
                       ,list_nfold=[0,1,2,3,4])

# make submission-file
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv("./home-credit-default-risk/submit/submission_HyperParameterTuning.csv", index=None)

In [ ]:
del df_train
del df_test
gc.collect()